# Library

In [1]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt
import random


import os
import time
import datetime
import calendar

from sklearn.metrics import mean_squared_error


import sklearn
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.model_selection import KFold, StratifiedKFold


from sklearn import clone
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# from utils_model import * # expand later

In [2]:
# np.set_seed(0)

# Preprocess

In [3]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
# data['gender'] = data['gender'].astype('str')
# data_test['gender'] = data_test['gender'].astype('str')

# data['Achievement_above_100%_during3quartal'] = data['Achievement_above_100%_during3quartal'].astype(str)
# data_test['Achievement_above_100%_during3quartal'] = data_test['Achievement_above_100%_during3quartal'].astype(str)

data = data.rename(columns={'annual leave':'annual_leave'})
data_test = data_test.rename(columns={'annual leave':'annual_leave'})

data = data.rename(columns={'Last_achievement_%':'Last_achievement','marital_status_maried(Y/N)':'marital_status_maried',
                           'Achievement_above_100%_during3quartal':'Achievement_above_100_during3quartal'})
data_test = data_test.rename(columns={'Last_achievement_%':'Last_achievement','marital_status_maried(Y/N)':'marital_status_maried',
                           'Achievement_above_100%_during3quartal':'Achievement_above_100_during3quartal'})


data['gender_str'] = data['gender'].astype('str')
data_test['gender_str'] = data_test['gender'].astype('str')

data['Achievement_above_100_during3quartal_str'] = data['Achievement_above_100_during3quartal'].astype(str)
data_test['Achievement_above_100_during3quartal_str'] = data_test['Achievement_above_100_during3quartal'].astype(str)


data['person_level_ordinary'] = LE.fit_transform(data['person_level'])
data['job_level_ordinary'] = LE.fit_transform(data['job_level'])
data['Education_level_ordinary'] = LE.fit_transform(data['Education_level'])

data_test['person_level_ordinary'] = LE.fit_transform(data_test['person_level'])
data_test['job_level_ordinary'] = LE.fit_transform(data_test['job_level'])
data_test['Education_level_ordinary'] = LE.fit_transform(data_test['Education_level'])

drop = ['age', 'job_duration_from_training', 'person_level_ordinary', 'job_level']


data = data.drop(columns=drop)
data_test = data_test.drop(columns=drop)
# data = data[~data['Last_achievement_%'].isna()]
# data = data.drop(columns=['job_level'])
# data_test = data_test.drop(columns=['job_level'])





In [4]:
# get test data (for final evaluation)
X_train = data.drop(columns=['Best Performance'])
y_train = data['Best Performance']
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [5]:
# len(X_train),len(X_val)

In [6]:
num_cols = list(X_train.select_dtypes(exclude=['object']))
cat_cols = list(X_train.select_dtypes(include=['object']))
features = list(X_train.columns)
len(num_cols),len(cat_cols),len(features)

(16, 6, 22)

# FE

In [7]:
class Feature_Engineering:
    def __init__(self,parameters):
        self.parameters = parameters
        self.target = parameters['target']
    
    @staticmethod  
    def check_col(col):
        if len(col.split(' '))>1:
            col2 = '_'.join(col.split(' '))
        else:
            col2 = col
        return col2
    
    @staticmethod
    def get_bin(data,col,n_bin,mode='cut'):
        while True:
            try:
                if mode=='cut':
                    _,bin_dummy = pd.cut(data[col],n_bin,retbins=True)
                else:
                    _,bin_dummy = pd.qcut(data[col],n_bin,retbins=True)
            except:
                n_bin -= 1
                continue
            break
        return bin_dummy
        
    def fit(self,data_ori):
        target = self.target
        data = data_ori.copy()
        for param in self.parameters['bin_numer_qcut']:
            col = param[0]
            n_bin = param[1]
            bin_dummy = self.get_bin(data,col,n_bin,mode='qcut')
            bin_dummy[0] = bin_dummy[0]-0.001
            bin_dummy[-1] = np.inf
            setattr(self,f'{col}_bin_numer_qcut',bin_dummy)
        for param in self.parameters['bin_numer_cut']:
            col = param[0]
            n_bin = param[1]
            bin_dummy = self.get_bin(data,col,n_bin,mode='cut')
            bin_dummy[0] = bin_dummy[0]-0.001
            bin_dummy[-1] = np.inf
            setattr(self,f'{col}_bin_numer_cut',bin_dummy)
            
            
        for param in self.parameters['bin_add_categ_numer_bin_cut']:
            col = param[1]
            n_bin = param[2]
            bin_dummy = self.get_bin(data,col,n_bin,mode='cut')
            bin_dummy[0] = bin_dummy[0]-0.001
            bin_dummy[-1] = np.inf
            setattr(self,f'{col}_bin_cut_add_categ',bin_dummy)
            
        for param in self.parameters['bin_add_categ_numer_bin_qcut']:
            col = param[1]
            n_bin = param[2]
            bin_dummy = self.get_bin(data,col,n_bin,mode='qcut')
            bin_dummy[0] = bin_dummy[0]-0.001
            bin_dummy[-1] = np.inf
            setattr(self,f'{col}_bin_qcut_add_categ',bin_dummy)
        
        for param in self.parameters['bin_target_encoding_cut']:
            col = param[0]
            n_bin = param[1]
            bin_dummy = self.get_bin(data,col,n_bin,mode='cut')
            bin_dummy[0] = bin_dummy[0]-0.001
            bin_dummy[-1] = np.inf
            setattr(self,f'{col}_bin_cut',bin_dummy)
            
            data[f'{col}_bin_target_encoding_cut'] = pd.cut(data[col],bins=bin_dummy)
            data_dummy = data.groupby([f'{col}_bin_target_encoding_cut'])[target].mean().reset_index(drop=False)
            setattr(self,f'{col}_bin_target_encoding_cut',data_dummy)
            
        for param in self.parameters['bin_target_encoding_qcut']:
            col = param[0]
            n_bin = param[1]
            bin_dummy = self.get_bin(data,col,n_bin,mode='qcut')
            bin_dummy[0] = bin_dummy[0]-0.001
            bin_dummy[-1] = np.inf
            setattr(self,f'{col}_bin_qcut',bin_dummy)
            
            data[f'{col}_bin_target_encoding_qcut'] = pd.cut(data[col],bins=bin_dummy)
            data_dummy = data.groupby([f'{col}_bin_target_encoding_qcut'])[target].mean().reset_index(drop=False)
            setattr(self,f'{col}_bin_target_encoding_qcut',data_dummy)
           
        for param in self.parameters['bin_target_encoding_custom_bin']:
            col = param[0]
            bins = param[1]
            setattr(self,f'{col}_bin_custom_bin',bins)
            
            data[f'{col}_bin_target_encoding_custom_bin'] = pd.cut(data[col],bins=bins)
            data_dummy = data.groupby([f'{col}_bin_target_encoding_custom_bin'])[target].mean().reset_index(drop=False)
            setattr(self,f'{col}_bin_target_encoding_custom_bin',data_dummy)
        
        for param in self.parameters['categorical_mean_encoding']:
            col = param
            data[f'{col}_categorical_mean_encoding'] = data[col].copy().values
            data_dummy = data.groupby([f'{col}_categorical_mean_encoding'])[target].mean().reset_index(drop=False)
            setattr(self,f'{col}_categorical_mean_encoding',data_dummy)
        
        
        
        self.fit = True
        return data
        
    
    def transform(self,X,mode='train'):
        data = X.copy()
        target = self.target
        if mode!='train':
            target_encode = self.target
        else:
            target_encode = self.target +"_y"
            
        if self.fit==False:
            raise Exception("Fit to train data first")
        
        for param in self.parameters['bin_numer_qcut']:
            col = param[0]
            bin_dummy = eval(f'self.{col}_bin_numer_qcut')
            data[f'{col}_bin_numer_qcut'] = pd.cut(data[col],bins=bin_dummy).astype(str).values
        for param in self.parameters['bin_numer_cut']:
            col = param[0]
            bin_dummy = eval(f'self.{col}_bin_numer_cut')
            data[f'{col}_bin_numer_cut'] = pd.cut(data[col],bins=bin_dummy).astype(str).values
            
        for cols in self.parameters['bin_add_categ_numer_bin_cut']:
            col_add = cols[0] + '_' + cols[1]
            bin_dummy = eval(f'self.{cols[1]}_bin_cut_add_categ')
            data[f'{col_add}_bin_add_categ_numer_bin_cut'] = pd.cut(data[cols[1]],bins=bin_dummy).values
            data[f'{col_add}_bin_add_categ_numer_bin_cut'] = (data[cols[0]].astype(str)+'_' + data[f'{col_add}_bin_add_categ_numer_bin_cut'].astype(str)).values
            
        for cols in self.parameters['bin_add_categ_numer_bin_qcut']:
            col_add = cols[0] + '_' + cols[1]
            bin_dummy = eval(f'self.{cols[1]}_bin_qcut_add_categ')
            data[f'{col_add}_bin_add_categ_numer_bin_qcut'] = pd.cut(data[cols[1]],bins=bin_dummy).values
            data[f'{col_add}_bin_add_categ_numer_bin_qcut'] = (data[cols[0]].astype(str)+'_' + data[f'{col_add}_bin_add_categ_numer_bin_qcut'].astype(str)).values
        
        for param in self.parameters['bin_target_encoding_cut']:
            col = param[0]
            bin_dummy = eval(f'self.{col}_bin_cut')
            data_dummy = eval(f'self.{col}_bin_target_encoding_cut')
            data[f'{col}_bin_target_encoding_cut'] = pd.cut(data[col],bins=bin_dummy).values
            data[f'{col}_bin_target_encoding_cut'] = pd.merge(data,data_dummy,how='left',on=[f'{col}_bin_target_encoding_cut'])[f'{target_encode}'].values
        
        for param in self.parameters['bin_target_encoding_qcut']:
            col = param[0]
            bin_dummy = eval(f'self.{col}_bin_qcut')
            data_dummy = eval(f'self.{col}_bin_target_encoding_qcut')
            data[f'{col}_bin_target_encoding_qcut'] = pd.cut(data[col],bins=bin_dummy).values
            data[f'{col}_bin_target_encoding_qcut'] = pd.merge(data,data_dummy,how='left',on=[f'{col}_bin_target_encoding_qcut'])[f'{target_encode}'].values
        
        for param in self.parameters['bin_target_encoding_custom_bin']:
            col = param[0]
            bin_dummy = eval(f'self.{col}_bin_custom_bin')
            data_dummy = eval(f'self.{col}_bin_target_encoding_custom_bin')
            data[f'{col}_bin_target_encoding_custom_bin'] = pd.cut(data[col],bins=bin_dummy).values
            data[f'{col}_bin_target_encoding_custom_bin'] = pd.merge(data,data_dummy,how='left',on=[f'{col}_bin_target_encoding_custom_bin'])[f'{target_encode}'].values
        
        for param in self.parameters['categorical_mean_encoding']:
            col = param
            data_dummy = eval(f'self.{col}_categorical_mean_encoding')
            data[f'{col}_categorical_mean_encoding'] = data[col].copy().values
            data[f'{col}_categorical_mean_encoding'] = pd.merge(data,data_dummy,how='left',on=[f'{col}_categorical_mean_encoding'])[f'{target_encode}'].values
        
        
        for cols in self.parameters['multiply']:
            data[cols[0] + '_times_' +cols[1]] = (data[cols[0]] * data[cols[1]]).values
        for cols in self.parameters['add']:
            data[cols[0] + '_plus_' +cols[1]] = (data[cols[0]] + data[cols[1]]).values
        for cols in self.parameters['add_str']:
            data[cols[0] + '_plus_' +cols[1]] = (data[cols[0]].astype(str)+'_' + data[cols[1]].astype(str)).values
            
        for cols in self.parameters['substract']:
            data[cols[0] + '_minus_' +cols[1]] = (data[cols[0]] - data[cols[1]]).values
        for cols in self.parameters['divide']:
            data[cols[0] + '_divide_' +cols[1]] = (data[cols[0]] / np.where(data[cols[1]]==0,0.0001,data[cols[1]])).values
        
        
        
        
        return data
    
    

## Model with FE

In [8]:
def fast_build_model_FE(X,y,cv,Feature_Engineering,parameters,model_base=LogisticRegression(class_weight='balanced')):

    num_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy = 'median')),
                                    ('scaler', RobustScaler())
                                    ])

    cat_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                    ])

    


    
    
    skf = StratifiedKFold(n_splits=cv,random_state = 3,shuffle = True)


    # oof validation
    oof_y_valid = []
    oof_y_valid_pred = []
    oof_y_valid_pred_proba = []
    pipelines = []
    add_fes = []
    data = pd.concat([X,y],axis=1)
    aucs=[]
#     print(data.columns)
    for cv,(train_index, val_index) in enumerate(skf.split(X,y)):
        start_fit = time.time()
        data_train = data.iloc[train_index,:].copy()
#         data_val = data.iloc[val_index,:][features]
        
        add_fe = Feature_Engineering(parameters)
        add_fe.fit(data_train)
        
        X_train = add_fe.transform(data_train).drop(columns=[parameters['target']])
        num_cols_fe = list(X_train.select_dtypes(exclude='object').columns)
        cat_cols_fe = list(X_train.select_dtypes(include='object').columns)
        
        print(X_train.shape,data.shape)
        y_train = y.iloc[train_index]
        
        X_val = add_fe.transform(X.iloc[val_index,:],mode='val')
        y_val = y.iloc[val_index]
        print(X_val.shape,data.shape)
        
        transformer = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_cols_fe),
            ('cat', cat_transformer, cat_cols_fe)
        ])
        
        main_pipeline = Pipeline(steps=[('transformer', transformer),
                          ('classifier', model_base)])
        

        
        add_fes.append(add_fe)
        model = clone(main_pipeline)
        model.fit(X_train,y_train.values.ravel())
        pred = model.predict(X_val)
        pred_proba = model.predict_proba(X_val)[:,1]
        oof_y_valid_pred.extend(pred)
        oof_y_valid_pred_proba.extend(pred_proba)
        oof_y_valid.extend(y_val.values)
        aucs.append(roc_auc_score(y_val.values, pred_proba,average='weighted'))
        pipelines.append(model)
        print(f'Fit iteration {cv} done in : {str(time.time()-start_fit)}')

    prec,rec,f1, _ = precision_recall_fscore_support(oof_y_valid,oof_y_valid_pred)
    auc = roc_auc_score(oof_y_valid, oof_y_valid_pred_proba,average='weighted')
    print(f'PRec Rec AUC average : {prec} {rec} <==> {auc}')
    print(aucs)
    print(np.mean(aucs[:2]))
    return add_fes,pipelines


def fast_predict_FE(data,add_fes,pipelines):
    X = data.copy()
#     pred = np.zeros(1,len(X))
    pred_proba = np.zeros((len(X)))
    for i in range(len(pipelines)):
        
        pred_proba += pipelines[i].predict_proba(add_fes[i].transform(X,mode='test'))[:,1] / len(pipelines)
    
    return pred_proba
        

In [99]:
num_cols

['job_duration_in_current_job_level',
 'job_duration_in_current_person_level',
 'job_duration_in_current_branch',
 'age',
 'number_of_dependences',
 'GPA',
 'year_graduated',
 'job_duration_from_training',
 'branch_rotation',
 'job_rotation',
 'assign_of_otherposition',
 'annual_leave',
 'sick_leaves',
 'Last_achievement']

In [100]:
cat_cols

['job_level',
 'person_level',
 'Employee_type',
 'gender',
 'marital_status_maried(Y/N)',
 'Education_level',
 'Achievement_above_100%_during3quartal']

In [16]:
data['job_duration_in_current_branch'].describe()

count    11153.000000
mean         1.034646
std          0.416723
min          0.000000
25%          0.707107
50%          1.118034
75%          1.224745
max          2.677686
Name: job_duration_in_current_branch, dtype: float64

In [327]:
parameters = {'multiply':[['GPA','number_of_dependences']],
              'add':[['annual_leave','sick_leaves'],['assign_of_otherposition','branch_rotation']],
              'add_str':[['Education_level','job_level']],
              'substract':[],'divide':[],
              'bin_numer_qcut':[],
              'bin_numer_cut':[],
              'bin_add_categ_numer_bin_qcut':[['job_level','GPA',5],['Education_level','GPA',5]],
            'bin_target_encoding_cut':[],
             'bin_target_encoding_qcut':[['year_graduated',5],['GPA',5],['annual_leave',5]],
             'bin_target_encoding_custom_bin':[],
              'categorical_mean_encoding':['job_level','person_level','Employee_type','Education_level'],
             'target':'Best Performance'}


# parameters = {'multiply':[['GPA','number_of_dependences']],
#               'add':[],
#               'add_str':[],
#               'substract':[],
#               'divide':[],
#               'bin_numer_qcut':[],
#               'bin_numer_cut':[],
#               'bin_add_categ_numer_bin_qcut':[],
#             'bin_target_encoding_cut':[],
#              'bin_target_encoding_qcut':[['year_graduated',5],['GPA',5],['annual_leave',5]],
#              'bin_target_encoding_custom_bin':[],
#               'categorical_mean_encoding':[],
#              'target':'Best Performance'}


# parameters = {'multiply':[],
#               'add':[],
#               'add_str':[],
#               'substract':[],
#               'divide':[],
#               'bin_numer_qcut':[],
#               'bin_numer_cut':[],
#               'bin_add_categ_numer_bin_qcut':[],
#             'bin_target_encoding_cut':[],
#              'bin_target_encoding_qcut':[],
#              'bin_target_encoding_custom_bin':[],
#               'categorical_mean_encoding':[],
#              'target':'Best Performance'}


### Logreg

In [33]:
# parameters = {'multiply':[['GPA','number_of_dependences']],
#               'add':[['annual_leave','sick_leaves'],['assign_of_otherposition','branch_rotation']],
#               'add_str':[['Education_level','job_level']],
#               'substract':[],'divide':[],
#               'bin_numer_qcut':[],
#               'bin_numer_cut':[['GPA',30]],
#               'bin_add_categ_numer_bin_qcut':[['job_level','GPA',5],['Education_level','GPA',5]],
#             'bin_target_encoding_cut':[],
#              'bin_target_encoding_qcut':[['year_graduated',5],['GPA',5],['annual_leave',5]],
#              'bin_target_encoding_custom_bin':[],
#               'categorical_mean_encoding':['job_level','person_level','Employee_type','Education_level'],
#              'target':'Best Performance'}


parameters = {'multiply':[['job_duration_in_current_job_level','number_of_dependences'],
                         ['job_duration_in_current_job_level','job_rotation'],
                         ['job_duration_in_current_job_level','job_level_ordinary'],
                         ['gender','GPA'],['gender','year_graduated'],['gender','sick_leaves'],['gender','job_level_ordinary'],
                         ['number_of_dependences','year_graduated'],['number_of_dependences','annual_leave'],['number_of_dependences','job_level_ordinary'],
                         ['GPA','branch_rotation'],['year_graduated','job_level_ordinary']],
              'add':[],
              'add_str':[['person_level','Education_level'],['Employee_type','Education_level']],
              'substract':[],
              'divide':[],
              'bin_numer_qcut':[['assign_of_otherposition',10],['Last_achievement',10]],
              'bin_numer_cut':[['Last_achievement',20]],
              'bin_add_categ_numer_bin_cut':[],
              'bin_add_categ_numer_bin_qcut':[],
            'bin_target_encoding_cut':[['job_duration_in_current_job_level',10],['assign_of_otherposition',20]],
             'bin_target_encoding_qcut':[['job_duration_in_current_branch',10]
                                        ],
             'bin_target_encoding_custom_bin':[],
              'categorical_mean_encoding':[],
             'target':'Best Performance'}


In [34]:
num_cols

['job_duration_in_current_job_level',
 'job_duration_in_current_person_level',
 'job_duration_in_current_branch',
 'gender',
 'number_of_dependences',
 'GPA',
 'year_graduated',
 'branch_rotation',
 'job_rotation',
 'assign_of_otherposition',
 'annual_leave',
 'sick_leaves',
 'Last_achievement',
 'Achievement_above_100_during3quartal',
 'job_level_ordinary',
 'Education_level_ordinary']

In [39]:
cv=5
add_fes,pipelines = fast_build_model_FE(X_train,y_train,cv,
                    Feature_Engineering,parameters,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=3000,C=0.001))

(8922, 42) (11153, 23)
(2231, 42) (11153, 23)
Fit iteration 0 done in : 0.875169038772583
(8922, 42) (11153, 23)
(2231, 42) (11153, 23)
Fit iteration 1 done in : 0.7901849746704102
(8922, 42) (11153, 23)
(2231, 42) (11153, 23)
Fit iteration 2 done in : 0.8934223651885986
(8923, 42) (11153, 23)
(2230, 42) (11153, 23)
Fit iteration 3 done in : 0.8351471424102783
(8923, 42) (11153, 23)
(2230, 42) (11153, 23)
Fit iteration 4 done in : 0.8423306941986084
PRec Rec AUC average : [0.87365591 0.17747364] [0.6147541  0.48320098] <==> 0.5650265777497719
[0.5803876596510164, 0.5668729092703434, 0.571831383053715, 0.5822723174900085, 0.5303327596375271]
0.5736302844606799


In [431]:
pred_proba = fast_predict_FE(data_test.copy(),add_fes,pipelines)

df_submission = pd.DataFrame({'index':data_test.index,'Best Performance':pred_proba})
df_submission

df_submission.to_csv('df_submission_20feb_LOGREG5CV057968_FE.csv',index=False)

In [430]:
# params = {'C': 4.599171011731616,
#  'class_weight': {0.0: 0.19089603168342784, 1.0: 1.938865730782311},
#  'random_state': 0,
#  'max_iter': 10000}
# cv=5
# add_fes,pipelines = fast_build_model_FE(X_train,y_train,cv,
#                     Feature_Engineering,parameters,model_base=LogisticRegression(**params))

In [432]:
df_submission

,index,Best Performance
0,0,0.429413
1,1,0.536009
2,2,0.649043
3,3,0.410727
4,4,0.580435
...,...,...
5995,5995,0.455349
5996,5996,0.454197
5997,5997,0.400584
5998,5998,0.408598


### SVC

In [420]:
parameters = {'multiply':[['job_duration_in_current_job_level','number_of_dependences'],
                         ['job_duration_in_current_job_level','job_rotation'],
                         ['job_duration_in_current_job_level','job_level_ordinary'],
                         ['gender','GPA'],['gender','year_graduated'],['gender','sick_leaves'],['gender','job_level_ordinary'],
                         ['number_of_dependences','year_graduated'],['number_of_dependences','annual_leave'],['number_of_dependences','job_level_ordinary'],
                         ['GPA','branch_rotation'],['year_graduated','job_level_ordinary']],
              'add':[],
              'add_str':[['person_level','Education_level'],['Employee_type','Education_level']],
              'substract':[],
              'divide':[],
              'bin_numer_qcut':[['assign_of_otherposition',10],['Last_achievement',10]],
              'bin_numer_cut':[['Last_achievement',20]],
              'bin_add_categ_numer_bin_qcut':[],
            'bin_target_encoding_cut':[['job_duration_in_current_job_level',10],['assign_of_otherposition',20]],
             'bin_target_encoding_qcut':[['job_duration_in_current_branch',10]
                                        ],
             'bin_target_encoding_custom_bin':[],
              'categorical_mean_encoding':[],
             'target':'Best Performance'}

In [423]:
cv=5
add_fes,pipelines = fast_build_model_FE(X_train,y_train,cv,
                    Feature_Engineering,parameters,model_base=SVC(class_weight='balanced',probability=True,kernel='linear'))

(8922, 42) (11153, 23)
(2231, 42) (11153, 23)
Fit iteration 0 done in : 389.66250944137573
(8922, 42) (11153, 23)
(2231, 42) (11153, 23)
Fit iteration 1 done in : 343.26217770576477
(8922, 42) (11153, 23)
(2231, 42) (11153, 23)
Fit iteration 2 done in : 360.49450063705444
(8923, 42) (11153, 23)
(2230, 42) (11153, 23)
Fit iteration 3 done in : 324.6629695892334
(8923, 42) (11153, 23)
(2230, 42) (11153, 23)
Fit iteration 4 done in : 321.88740038871765
PRec Rec AUC average : [0.8745357  0.18666323] [0.66803279 0.44288332] <==> 0.5799965103944795
[0.604585871045666, 0.5815224356920395, 0.5899910923701985, 0.5778418431544592, 0.5550016793056513]
0.5930541533688527


In [425]:
pred_proba = fast_predict_FE(data_test.copy(),add_fes,pipelines)

df_submission = pd.DataFrame({'index':data_test.index,'Best Performance':pred_proba})
df_submission

df_submission.to_csv('df_submission_20feb_SCV5CV05799_FE.csv',index=False)

In [426]:
df_submission

,index,Best Performance
0,0,0.125095
1,1,0.154628
2,2,0.210143
3,3,0.122740
4,4,0.171385
...,...,...
5995,5995,0.129253
5996,5996,0.132509
5997,5997,0.118102
5998,5998,0.122387


In [427]:
import pickle
with open("pipelines_05799_SCV.pkl", "wb") as f:
    pickle.dump(pipelines, f)

### Automatic FE

In [9]:
def fast_build_model_FE_1(X,y,Feature_Engineering,parameters,model_base=LogisticRegression(class_weight='balanced')):

    num_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy = 'median')),
                                    ('scaler', RobustScaler())
                                    ])

    cat_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                    ])
    data = pd.concat([X,y],axis=1)
    aucs=[]
    start_fit = time.time()
    data_train = data.copy()
    add_fe = Feature_Engineering(parameters)
    add_fe.fit(data_train)

    X_train = add_fe.transform(data_train).drop(columns=[parameters['target']])
    num_cols_fe = list(X_train.select_dtypes(exclude='object').columns)
    cat_cols_fe = list(X_train.select_dtypes(include='object').columns)

    y_train = y

    
    transformer = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols_fe),
        ('cat', cat_transformer, cat_cols_fe)
    ])

    main_pipeline = Pipeline(steps=[('transformer', transformer),
                      ('classifier', model_base)])



#     add_fes.append(add_fe)
    model = clone(main_pipeline)
    model.fit(X_train,y_train.values.ravel())
    
    return [add_fe],[model]


def fast_build_model_FE_2fe_1(X,y,Feature_Engineering,parameters1,parameters2,model_base=LogisticRegression(class_weight='balanced')):

    num_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy = 'median')),
                                    ('scaler', RobustScaler())
                                    ])

    cat_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                    ])
    data = pd.concat([X,y],axis=1)
    aucs=[]
    start_fit = time.time()
    data_train = data.copy()
    add_fe1 = Feature_Engineering(parameters1)
    add_fe2 = Feature_Engineering(parameters2)
    
    X_train = add_fe1.transform(data_train)
    add_fe2.fit(X_train)
    X_train = add_fe2.transform(X_train).drop(columns=[parameters1['target']])

    num_cols_fe = list(X_train.select_dtypes(exclude='object').columns)
    cat_cols_fe = list(X_train.select_dtypes(include='object').columns)
    y_train = y.iloc[train_index]


    transformer = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols_fe),
        ('cat', cat_transformer, cat_cols_fe)
    ])

    main_pipeline = Pipeline(steps=[('transformer', transformer),
                      ('classifier', model_base)])



    model = clone(main_pipeline)
    model.fit(X_train,y_train.values.ravel())
    
    
    return [add_fe1],[add_fe2],[model]



def fast_predict_FE(data,add_fes,pipelines):
    X = data.copy()
#     pred = np.zeros(1,len(X))
    pred_proba = np.zeros((len(X)))
    for i in range(len(pipelines)):
        
        pred_proba += pipelines[i].predict_proba(add_fes[i].transform(X,mode='test'))[:,1] / len(pipelines)
    
    return pred_proba
        

In [10]:
def fast_build_model_FE_automatic(X,y,cv,Feature_Engineering,parameters,model_base=LogisticRegression(class_weight='balanced')):

    num_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy = 'median')),
                                    ('scaler', RobustScaler())
                                    ])

    cat_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                    ])

    


    
    
    skf = StratifiedKFold(n_splits=cv,random_state = 3,shuffle = True)


    # oof validation
    oof_y_valid = []
    oof_y_valid_pred = []
    oof_y_valid_pred_proba = []
    pipelines = []
    add_fes = []
    data = pd.concat([X,y],axis=1)
    aucs=[]
#     print(data.columns)
    for cv,(train_index, val_index) in enumerate(skf.split(X,y)):
        start_fit = time.time()
        data_train = data.iloc[train_index,:].copy()
#         data_val = data.iloc[val_index,:][features]
        
        add_fe = Feature_Engineering(parameters)
        add_fe.fit(data_train)
        
        X_train = add_fe.transform(data_train).drop(columns=[parameters['target']])
        num_cols_fe = list(X_train.select_dtypes(exclude='object').columns)
        cat_cols_fe = list(X_train.select_dtypes(include='object').columns)
        

        y_train = y.iloc[train_index]
        
        X_val = add_fe.transform(X.iloc[val_index,:],mode='val')
        y_val = y.iloc[val_index]
        
        
        transformer = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_cols_fe),
            ('cat', cat_transformer, cat_cols_fe)
        ])
        
        main_pipeline = Pipeline(steps=[('transformer', transformer),
                          ('classifier', model_base)])
        

        
        add_fes.append(add_fe)
        model = clone(main_pipeline)
        model.fit(X_train,y_train.values.ravel())
        pred = model.predict(X_val)
        pred_proba = model.predict_proba(X_val)[:,1]
        oof_y_valid_pred.extend(pred)
        oof_y_valid_pred_proba.extend(pred_proba)
        oof_y_valid.extend(y_val.values)
        aucs.append(roc_auc_score(y_val.values, pred_proba,average='weighted'))
        pipelines.append(model)
#         print(f'Fit iteration {cv} done in : {str(time.time()-start_fit)}')

    prec,rec,f1, _ = precision_recall_fscore_support(oof_y_valid,oof_y_valid_pred)
    auc = roc_auc_score(oof_y_valid, oof_y_valid_pred_proba,average='weighted')
#     print(f'PRec Rec AUC average : {prec} {rec} <==> {auc}')
#     print(np.mean(aucs))
    return auc


def fast_predict_FE(data,add_fes,pipelines):
    X = data.copy()
#     pred = np.zeros(1,len(X))
    pred_proba = np.zeros((len(X)))
    for i in range(len(pipelines)):
        
        pred_proba += pipelines[i].predict_proba(add_fes[i].transform(X,mode='test'))[:,1] / len(pipelines)
    
    return pred_proba
        

In [12]:
def fast_build_model_FE_automatic_2(X,y,cv,Feature_Engineering,parameters1,parameters2,model_base=LogisticRegression(class_weight='balanced')):

    num_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy = 'median')),
                                    ('scaler', RobustScaler())
                                    ])

    cat_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                    ])

    


    
    
    skf = StratifiedKFold(n_splits=cv,random_state = 3,shuffle = True)


    # oof validation
    oof_y_valid = []
    oof_y_valid_pred = []
    oof_y_valid_pred_proba = []
    pipelines = []
    add_fes1 = []
    add_fes2 = []
    data = pd.concat([X,y],axis=1)
    aucs=[]
#     print(data.columns)
    for cv,(train_index, val_index) in enumerate(skf.split(X,y)):
        start_fit = time.time()
        data_train = data.iloc[train_index,:].copy()
#         data_val = data.iloc[val_index,:][features]
        
        add_fe1 = Feature_Engineering(parameters1)
        add_fe2 = Feature_Engineering(parameters2)
        add_fe1.fit(data_train)
        
        X_train = add_fe1.transform(data_train)
        add_fe2.fit(X_train)
        X_train = add_fe2.transform(X_train).drop(columns=[parameters1['target']])
        
        num_cols_fe = list(X_train.select_dtypes(exclude='object').columns)
        cat_cols_fe = list(X_train.select_dtypes(include='object').columns)
        

        y_train = y.iloc[train_index]
        
        X_val = add_fe1.transform(X.iloc[val_index,:],mode='val')
        X_val = add_fe2.transform(X_val,mode='val')
        y_val = y.iloc[val_index]
        
        
        transformer = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_cols_fe),
            ('cat', cat_transformer, cat_cols_fe)
        ])
        
        main_pipeline = Pipeline(steps=[('transformer', transformer),
                          ('classifier', model_base)])
        

        
        add_fes1.append(add_fe1)
        add_fes2.append(add_fe2)
        model = clone(main_pipeline)
        model.fit(X_train,y_train.values.ravel())
        pred = model.predict(X_val)
        pred_proba = model.predict_proba(X_val)[:,1]
        oof_y_valid_pred.extend(pred)
        oof_y_valid_pred_proba.extend(pred_proba)
        oof_y_valid.extend(y_val.values)
        aucs.append(roc_auc_score(y_val.values, pred_proba,average='weighted'))
        pipelines.append(model)
#         print(f'Fit iteration {cv} done in : {str(time.time()-start_fit)}')

    prec,rec,f1, _ = precision_recall_fscore_support(oof_y_valid,oof_y_valid_pred)
    auc = roc_auc_score(oof_y_valid, oof_y_valid_pred_proba,average='weighted')
#     print(f'PRec Rec AUC average : {prec} {rec} <==> {auc}')
#     print(np.mean(aucs))
    return auc


def fast_predict_FE_2(data,add_fes1,add_fes2,pipelines):
    X = data.copy()
#     pred = np.zeros(1,len(X))
    pred_proba = np.zeros((len(X)))
    for i in range(len(pipelines)):
        
        pred_proba += pipelines[i].predict_proba(add_fes2[i].transform(add_fes1[i].transform(X,mode='test'),mode='test'))[:,1] / len(pipelines)
    
    return pred_proba
        

In [13]:
def fast_build_model_FE_automatic_2_pipe(X,y,cv,Feature_Engineering,parameters1,parameters2,model_base=LogisticRegression(class_weight='balanced')):

    num_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy = 'median')),
                                    ('scaler', RobustScaler())
                                    ])

    cat_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                    ])

    


    
    
    skf = StratifiedKFold(n_splits=cv,random_state = 3,shuffle = True)


    # oof validation
    oof_y_valid = []
    oof_y_valid_pred = []
    oof_y_valid_pred_proba = []
    pipelines = []
    add_fes1 = []
    add_fes2 = []
    data = pd.concat([X,y],axis=1)
    aucs=[]
#     print(data.columns)
    for cv,(train_index, val_index) in enumerate(skf.split(X,y)):
        start_fit = time.time()
        data_train = data.iloc[train_index,:].copy()
#         data_val = data.iloc[val_index,:][features]
        
        add_fe1 = Feature_Engineering(parameters1)
        add_fe2 = Feature_Engineering(parameters2)
        add_fe1.fit(data_train)
        
        X_train = add_fe1.transform(data_train)
        add_fe2.fit(X_train)
        X_train = add_fe2.transform(X_train).drop(columns=[parameters1['target']])
        
        num_cols_fe = list(X_train.select_dtypes(exclude='object').columns)
        cat_cols_fe = list(X_train.select_dtypes(include='object').columns)
        

        y_train = y.iloc[train_index]
        
        X_val = add_fe1.transform(X.iloc[val_index,:],mode='val')
        X_val = add_fe2.transform(X_val,mode='val')
        y_val = y.iloc[val_index]
        
        
        transformer = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_cols_fe),
            ('cat', cat_transformer, cat_cols_fe)
        ])
        
        main_pipeline = Pipeline(steps=[('transformer', transformer),
                          ('classifier', model_base)])
        

        
        add_fes1.append(add_fe1)
        add_fes2.append(add_fe2)
        model = clone(main_pipeline)
        model.fit(X_train,y_train.values.ravel())
        pred = model.predict(X_val)
        pred_proba = model.predict_proba(X_val)[:,1]
        oof_y_valid_pred.extend(pred)
        oof_y_valid_pred_proba.extend(pred_proba)
        oof_y_valid.extend(y_val.values)
        aucs.append(roc_auc_score(y_val.values, pred_proba,average='weighted'))
        pipelines.append(model)
        print(f'Fit iteration {cv} done in : {str(time.time()-start_fit)}')

    prec,rec,f1, _ = precision_recall_fscore_support(oof_y_valid,oof_y_valid_pred)
    auc = roc_auc_score(oof_y_valid, oof_y_valid_pred_proba,average='weighted')
    print(f'PRec Rec AUC average : {prec} {rec} <==> {auc}')
    print(np.mean(aucs))
    return add_fes1,add_fes2,pipelines





In [14]:
# X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.1, random_state=0,shuffle=True)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.5, random_state=1,shuffle=True)

In [ ]:
import copy

cv=2

parameters = {'multiply':[],
              'add':[],
              'add_str':[],
              'substract':[],'divide':[],
              'bin_numer_qcut':[],
              'bin_numer_cut':[],
              'bin_add_categ_numer_bin_cut':[],
              'bin_add_categ_numer_bin_qcut':[],
            'bin_target_encoding_cut':[],
             'bin_target_encoding_qcut':[],
             'bin_target_encoding_custom_bin':[],
              'categorical_mean_encoding':[],
             'target':'Best Performance'}



auc_end = 0

for i,param in enumerate(parameters):
    if i in [0,1,3,4]:
        print(param)
        for j in range(len(num_cols)):
            if j!=len(num_cols)-1:
                for k in range(len(num_cols[j+1:])):
                    parameters2 = copy.deepcopy(parameters)
                    col_col = [num_cols[j],num_cols[j+1+k]]
                    if col_col in parameters2[param]:
                        continue
                    parameters2[param].append(col_col)
    #                 print(parameters)
    #                 print(parameters2)
    #                 print()

                    try:
                        auc = fast_build_model_FE_automatic(X_train2,y_train2,cv,
                                        Feature_Engineering,parameters2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.01))
                    except ZeroDivisionError:
                        continue
                    except AttributeError:
                        continue
                        
                    if auc>auc_end+0.004:
                        auc_end = auc.copy()
                        parameters[param].append(col_col)
                        print(parameters[param][-1],auc)
                    else:
    #                     print(parameters)
                        continue
    elif i in [2]:
        print(param)
        for j in range(len(cat_cols)):
            if j!=len(cat_cols)-1:
                for k in range(len(cat_cols[j+1:])):
                    parameters2 = copy.deepcopy(parameters)
                    col_col = [cat_cols[j],cat_cols[j+1+k]]
                    if col_col in parameters2[param]:
                        continue
                    parameters2[param].append(col_col)
    #                 print(parameters)
    #                 print(parameters2)
    #                 print()

                    try:
                        auc = fast_build_model_FE_automatic(X_train2,y_train2,cv,
                                        Feature_Engineering,parameters2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.01))
                    except ZeroDivisionError:
                        continue
                    except AttributeError:
                        continue
                        
                    if auc>auc_end+0.004:
                        auc_end = auc.copy()
                        parameters[param].append(col_col)
                        print(parameters[param][-1],auc)
                    else:
    #                     print(parameters)
                        continue
    elif i in [5,6,9,10]:
        print(param)
        for j in range(len(num_cols)):
            if j!=len(num_cols)-1:
                bins = [5,10,20]
                for bin_ in bins:
                    parameters2 = copy.deepcopy(parameters)
                    col_col = [num_cols[j],bin_]
                    if col_col in parameters2[param]:
                        continue
                    parameters2[param].append(col_col)
    #                 print(parameters)
    #                 print(parameters2)
    #                 print()
                    
                    try:
                        auc = fast_build_model_FE_automatic(X_train2,y_train2,cv,
                                        Feature_Engineering,parameters2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.01))
                    except ZeroDivisionError:
                        continue
                    except AttributeError:
                        continue
                    if auc>auc_end+0.004:
                        auc_end = auc.copy()
                        parameters[param].append(col_col)
                        print(parameters[param][-1],auc)
                    else:
    #                     print(parameters)
                        continue
        
    elif i in [7,8]:
        print(param)
        for j in range(len(num_cols)):
            if j!=len(num_cols)-1:
                bins = [10]
                for bin_ in bins:
                    for col2 in cat_cols:
                        parameters2 = copy.deepcopy(parameters)
                        col_col = [col2,num_cols[j],bin_]
                        if col_col in parameters2[param]:
                            continue
                        parameters2[param].append(col_col)
        #                 print(parameters)
        #                 print(parameters2)
        #                 print()

                        try:
                            auc = fast_build_model_FE_automatic(X_train2,y_train2,cv,
                                            Feature_Engineering,parameters2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.01))
                        except ZeroDivisionError:
                            continue
                        except AttributeError:
                            continue
                        if auc>auc_end+0.004:
                            auc_end = auc.copy()
                            parameters[param].append(col_col)
                            print(parameters[param][-1],auc)
                        else:
        #                     print(parameters)
                            continue
        
    elif i in [12]:
        print(param)
        for j in range(len(cat_cols)):
            if j!=len(cat_cols)-1:
                parameters2 = copy.deepcopy(parameters)
                col_col = cat_cols[j]
                if col_col in parameters2[param]:
                    continue
                parameters2[param].append(col_col)
                
#                 print(parameters)
#                 print(parameters2)
#                 print()
                
                try:
                    auc = fast_build_model_FE_automatic(X_train2,y_train2,cv,
                                    Feature_Engineering,parameters2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.01))
                except ZeroDivisionError:
                    continue
                except AttributeError:
                    continue
                if auc>auc_end+0.004:
                    auc_end = auc.copy()
                    parameters[param].append(col_col)
                    print(parameters[param][-1],auc)
                else:
#                     print(parameters)
                    continue
    print('=========================\n')



multiply
['job_duration_in_current_job_level', 'job_duration_in_current_person_level'] 0.5172544790216418
['number_of_dependences', 'assign_of_otherposition'] 0.5227792484525233

add

add_str

substract

divide
['job_duration_in_current_job_level', 'Last_achievement'] 0.5274137694374178
['number_of_dependences', 'sick_leaves'] 0.5326454497323372

bin_numer_qcut
['job_duration_in_current_branch', 5] 0.5473867090220993

bin_numer_cut
['job_duration_in_current_branch', 5] 0.5545860867632737
['Last_achievement', 5] 0.5590049087854843

bin_add_categ_numer_bin_cut
['Employee_type', 'job_duration_in_current_job_level', 10] 0.5631546953742377

bin_add_categ_numer_bin_qcut


In [73]:
# parameters # 5.85 --> 5.66
# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_branch', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'Achievement_above_100_during3quartal'],
#   ['number_of_dependences', 'assign_of_otherposition'],
#   ['assign_of_otherposition', 'sick_leaves'],
#   ['Last_achievement', 'Achievement_above_100_during3quartal']],
#  'add': [['gender', 'number_of_dependences']],
#  'add_str': [],
#  'substract': [],
#  'divide': [['job_duration_in_current_job_level', 'sick_leaves'],
#   ['job_duration_in_current_job_level', 'Last_achievement'],
#   ['job_rotation', 'assign_of_otherposition']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5],
#   ['number_of_dependences', 5],
#   ['Last_achievement', 5]],
#  'bin_add_categ_numer_bin_cut': [['Employee_type',
#    'job_duration_in_current_job_level',
#    10],
#   ['Education_level', 'branch_rotation', 10]],
#  'bin_add_categ_numer_bin_qcut': [['Employee_type', 'Last_achievement', 10]],
#  'bin_target_encoding_cut': [['job_duration_in_current_branch', 20],
#   ['year_graduated', 10]],
#  'bin_target_encoding_qcut': [],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}

In [65]:
# parameters # 5.78 --> 5.58
# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_branch', 'assign_of_otherposition'],
#   ['number_of_dependences', 'assign_of_otherposition'],
#   ['assign_of_otherposition', 'sick_leaves']],
#  'add': [],
#  'add_str': [],
#  'substract': [],
#  'divide': [['job_duration_in_current_job_level', 'Last_achievement'],
#   ['gender', 'sick_leaves'],
#   ['number_of_dependences', 'sick_leaves'],
#   ['job_rotation', 'assign_of_otherposition']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5],
#   ['number_of_dependences', 5],
#   ['Last_achievement', 5]],
#  'bin_add_categ_numer_bin_cut': [],
#  'bin_add_categ_numer_bin_qcut': [['Employee_type', 'Last_achievement', 10]],
#  'bin_target_encoding_cut': [['year_graduated', 10]],
#  'bin_target_encoding_qcut': [],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}

In [66]:
# parameters # 5.97 --> 5.72
# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_job_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_person_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'Achievement_above_100_during3quartal'],
#   ['number_of_dependences', 'assign_of_otherposition'],
#   ['number_of_dependences', 'Last_achievement'],
#   ['assign_of_otherposition', 'sick_leaves'],
#   ['Last_achievement', 'Achievement_above_100_during3quartal']],
#  'add': [['gender', 'number_of_dependences']],
#  'add_str': [['person_level', 'gender_str'],
#   ['marital_status_maried', 'gender_str']],
#  'substract': [],
#  'divide': [['job_duration_in_current_job_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_job_level', 'sick_leaves'],
#   ['job_duration_in_current_job_level', 'Last_achievement'],
#   ['gender', 'sick_leaves'],
#   ['branch_rotation', 'Achievement_above_100_during3quartal'],
#   ['job_rotation', 'assign_of_otherposition'],
#   ['Achievement_above_100_during3quartal', 'Education_level_ordinary']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5],
#   ['Last_achievement', 10]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5],
#   ['number_of_dependences', 5],
#   ['branch_rotation', 5],
#   ['Last_achievement', 5]],
#  'bin_add_categ_numer_bin_cut': [['Employee_type',
#    'job_duration_in_current_job_level',
#    10],
#   ['marital_status_maried', 'job_duration_in_current_branch', 10],
#   ['Education_level', 'year_graduated', 10],
#   ['Employee_type', 'branch_rotation', 10],
#   ['Education_level', 'branch_rotation', 10],
#   ['gender_str', 'job_rotation', 10],
#   ['Employee_type', 'annual_leave', 10],
#   ['Employee_type', 'Last_achievement', 10]],
#  'bin_add_categ_numer_bin_qcut': [['Education_level',
#    'job_duration_in_current_branch',
#    10],
#   ['Achievement_above_100_during3quartal_str', 'year_graduated', 10],
#   ['Employee_type', 'Last_achievement', 10]],
#  'bin_target_encoding_cut': [['job_duration_in_current_branch', 20],
#   ['job_rotation', 5]],
#  'bin_target_encoding_qcut': [],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}

In [54]:
# parameters  # 5.64 --> 5.65
# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['number_of_dependences', 'assign_of_otherposition']],
#  'add': [],
#  'add_str': [],
#  'substract': [],
#  'divide': [['job_duration_in_current_person_level', 'Last_achievement'],
#   ['number_of_dependences', 'sick_leaves']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5]],
#  'bin_add_categ_numer_bin_cut': [],
#  'bin_add_categ_numer_bin_qcut': [['Employee_type', 'Last_achievement', 10]],
#  'bin_target_encoding_cut': [['job_rotation', 20]],
#  'bin_target_encoding_qcut': [],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}

In [29]:
# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level']],
#  'add': [],
#  'add_str': [],
#  'substract': [],
#  'divide': [['job_duration_in_current_job_level', 'branch_rotation']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5]],
#  'bin_add_categ_numer_bin_cut': [],
#  'bin_add_categ_numer_bin_qcut': [['marital_status_maried',
#    'branch_rotation',
#    10]],
#  'bin_target_encoding_cut': [['job_duration_in_current_branch', 20]],
#  'bin_target_encoding_qcut': [],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}


# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['job_duration_in_current_job_level', 'number_of_dependences'],
#   ['job_duration_in_current_job_level', 'job_rotation'],
#   ['job_duration_in_current_branch', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'Achievement_above_100_during3quartal'],
#   ['gender', 'assign_of_otherposition'],
#   ['gender', 'Achievement_above_100_during3quartal'],
#   ['number_of_dependences', 'job_rotation'],
#   ['number_of_dependences', 'assign_of_otherposition'],
#   ['GPA', 'branch_rotation'],
#   ['branch_rotation', 'job_rotation'],
#   ['assign_of_otherposition', 'sick_leaves'],
#   ['assign_of_otherposition', 'Education_level_ordinary'],
#   ['annual_leave', 'Last_achievement'],
#   ['Last_achievement', 'Achievement_above_100_during3quartal']],
#  'add': [['number_of_dependences', 'job_level_ordinary']],
#  'add_str': [['person_level', 'gender_str'],
#   ['Employee_type', 'Education_level'],
#   ['Employee_type', 'Achievement_above_100_during3quartal_str'],
#   ['marital_status_maried', 'Education_level'],
#   ['marital_status_maried', 'Achievement_above_100_during3quartal_str']],
#  'substract': [],
#  'divide': [['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_job_level', 'sick_leaves'],
#   ['job_duration_in_current_branch', 'sick_leaves'],
#   ['job_duration_in_current_branch', 'Last_achievement'],
#   ['job_duration_in_current_branch', 'Achievement_above_100_during3quartal'],
#   ['gender', 'Last_achievement'],
#   ['number_of_dependences', 'Last_achievement'],
#   ['year_graduated', 'Last_achievement'],
#   ['annual_leave', 'Achievement_above_100_during3quartal'],
#   ['Achievement_above_100_during3quartal', 'Education_level_ordinary']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5],
#   ['branch_rotation', 5],
#   ['Last_achievement', 20]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5],
#   ['job_duration_in_current_branch', 20],
#   ['year_graduated', 5],
#   ['assign_of_otherposition', 20]],
#  'bin_add_categ_numer_bin_cut': [['Employee_type',
#    'job_duration_in_current_job_level',
#    10],
#   ['Achievement_above_100_during3quartal_str', 'number_of_dependences', 10],
#   ['marital_status_maried', 'branch_rotation', 10],
#   ['Education_level', 'branch_rotation', 10],
#   ['marital_status_maried', 'sick_leaves', 10]],
#  'bin_add_categ_numer_bin_qcut': [['person_level', 'branch_rotation', 10],
#   ['marital_status_maried', 'branch_rotation', 10],
#   ['Achievement_above_100_during3quartal_str', 'branch_rotation', 10],
#   ['gender_str', 'Last_achievement', 10],
#   ['Achievement_above_100_during3quartal_str', 'Last_achievement', 10]],
#  'bin_target_encoding_cut': [['job_duration_in_current_branch', 5],
#   ['job_duration_in_current_branch', 20],
#   ['job_rotation', 5]],
#  'bin_target_encoding_qcut': [],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}


# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_job_level', 'job_rotation'],
#   ['job_duration_in_current_job_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_person_level', 'number_of_dependences'],
#   ['job_duration_in_current_person_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'year_graduated'],
#   ['job_duration_in_current_branch', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'annual_leave'],
#   ['job_duration_in_current_branch', 'Achievement_above_100_during3quartal'],
#   ['gender', 'sick_leaves'],
#   ['gender', 'Achievement_above_100_during3quartal'],
#   ['number_of_dependences', 'assign_of_otherposition'],
#   ['number_of_dependences', 'Last_achievement'],
#   ['branch_rotation', 'job_level_ordinary'],
#   ['job_rotation', 'annual_leave'],
#   ['job_rotation', 'sick_leaves'],
#   ['job_rotation', 'job_level_ordinary'],
#   ['assign_of_otherposition', 'sick_leaves'],
#   ['assign_of_otherposition', 'Achievement_above_100_during3quartal'],
#   ['annual_leave', 'sick_leaves'],
#   ['sick_leaves', 'job_level_ordinary'],
#   ['Last_achievement', 'Achievement_above_100_during3quartal']],
#  'add': [['job_duration_in_current_job_level', 'job_level_ordinary']],
#  'add_str': [['person_level', 'gender_str'],
#   ['marital_status_maried', 'gender_str'],
#   ['marital_status_maried', 'Achievement_above_100_during3quartal_str']],
#  'substract': [['number_of_dependences',
#    'Achievement_above_100_during3quartal']],
#  'divide': [['job_duration_in_current_job_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_job_level', 'sick_leaves'],
#   ['job_duration_in_current_job_level', 'Last_achievement'],
#   ['job_duration_in_current_branch', 'Last_achievement'],
#   ['gender', 'sick_leaves'],
#   ['number_of_dependences', 'sick_leaves'],
#   ['GPA', 'Last_achievement'],
#   ['branch_rotation', 'Achievement_above_100_during3quartal'],
#   ['job_rotation', 'assign_of_otherposition'],
#   ['job_rotation', 'Last_achievement'],
#   ['Achievement_above_100_during3quartal', 'job_level_ordinary'],
#   ['Achievement_above_100_during3quartal', 'Education_level_ordinary']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5],
#   ['GPA', 5],
#   ['Last_achievement', 10]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5],
#   ['number_of_dependences', 5],
#   ['branch_rotation', 5],
#   ['job_rotation', 5],
#   ['Last_achievement', 5]],
#  'bin_add_categ_numer_bin_cut': [['Employee_type',
#    'job_duration_in_current_job_level',
#    10],
#   ['Education_level', 'job_duration_in_current_job_level', 10],
#   ['marital_status_maried', 'job_duration_in_current_branch', 10],
#   ['Education_level', 'job_duration_in_current_branch', 10],
#   ['Achievement_above_100_during3quartal_str',
#    'job_duration_in_current_branch',
#    10],
#   ['Education_level', 'year_graduated', 10],
#   ['Employee_type', 'branch_rotation', 10],
#   ['Education_level', 'branch_rotation', 10],
#   ['gender_str', 'job_rotation', 10],
#   ['Employee_type', 'annual_leave', 10],
#   ['Employee_type', 'Last_achievement', 10]],
#  'bin_add_categ_numer_bin_qcut': [['marital_status_maried',
#    'job_duration_in_current_branch',
#    10],
#   ['Education_level', 'year_graduated', 10],
#   ['Achievement_above_100_during3quartal_str', 'year_graduated', 10],
#   ['Employee_type', 'Last_achievement', 10],
#   ['marital_status_maried', 'Last_achievement', 10]],
#  'bin_target_encoding_cut': [['job_duration_in_current_branch', 20],
#   ['year_graduated', 10],
#   ['job_rotation', 5]],
#  'bin_target_encoding_qcut': [],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}

In [30]:
# parameters_backup = {'multiply': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['job_duration_in_current_job_level', 'job_duration_in_current_branch'],
#   ['job_duration_in_current_job_level', 'gender'],
#   ['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_job_level', 'sick_leaves'],
#   ['job_duration_in_current_job_level', 'Last_achievement'],
#   ['job_duration_in_current_job_level', 'job_level_ordinary'],
#   ['job_duration_in_current_job_level', 'Education_level_ordinary'],
#   ['job_duration_in_current_person_level', 'job_duration_in_current_branch'],
#   ['job_duration_in_current_person_level', 'gender'],
#   ['job_duration_in_current_person_level', 'branch_rotation'],
#   ['job_duration_in_current_person_level', 'job_rotation'],
#   ['job_duration_in_current_branch', 'gender'],
#   ['job_duration_in_current_branch', 'number_of_dependences'],
#   ['job_duration_in_current_branch', 'GPA'],
#   ['job_duration_in_current_branch', 'year_graduated'],
#   ['job_duration_in_current_branch', 'branch_rotation'],
#   ['job_duration_in_current_branch', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'Achievement_above_100_during3quartal'],
#   ['job_duration_in_current_branch', 'Education_level_ordinary'],
#   ['gender', 'number_of_dependences'],
#   ['gender', 'GPA'],
#   ['gender', 'sick_leaves'],
#   ['gender', 'Last_achievement'],
#   ['gender', 'job_level_ordinary'],
#   ['number_of_dependences', 'GPA'],
#   ['number_of_dependences', 'assign_of_otherposition'],
#   ['number_of_dependences', 'Last_achievement'],
#   ['number_of_dependences', 'Education_level_ordinary'],
#   ['GPA', 'year_graduated'],
#   ['year_graduated', 'assign_of_otherposition'],
#   ['year_graduated', 'job_level_ordinary'],
#   ['branch_rotation', 'assign_of_otherposition'],
#   ['branch_rotation', 'job_level_ordinary'],
#   ['assign_of_otherposition', 'sick_leaves'],
#   ['sick_leaves', 'Last_achievement'],
#   ['sick_leaves', 'Education_level_ordinary'],
#   ['job_level_ordinary', 'Education_level_ordinary']],
#  'add': [['job_duration_in_current_job_level', 'number_of_dependences'],
#   ['job_duration_in_current_job_level', 'GPA'],
#   ['job_duration_in_current_job_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_job_level', 'job_level_ordinary'],
#   ['job_duration_in_current_person_level', 'job_duration_in_current_branch'],
#   ['job_duration_in_current_person_level', 'number_of_dependences'],
#   ['job_duration_in_current_branch', 'GPA'],
#   ['job_duration_in_current_branch', 'job_level_ordinary'],
#   ['gender', 'number_of_dependences'],
#   ['gender', 'GPA'],
#   ['gender', 'branch_rotation'],
#   ['number_of_dependences', 'assign_of_otherposition'],
#   ['number_of_dependences', 'Education_level_ordinary'],
#   ['GPA', 'Last_achievement'],
#   ['GPA', 'job_level_ordinary']],
#  'add_str': [['person_level', 'Employee_type'],
#   ['person_level', 'gender_str'],
#   ['person_level', 'Achievement_above_100_during3quartal_str'],
#   ['Employee_type', 'marital_status_maried'],
#   ['Employee_type', 'Education_level'],
#   ['marital_status_maried', 'gender_str'],
#   ['marital_status_maried', 'Achievement_above_100_during3quartal_str'],
#   ['Education_level', 'Achievement_above_100_during3quartal_str']],
#  'substract': [['job_duration_in_current_job_level',
#    'job_duration_in_current_person_level'],
#   ['job_duration_in_current_job_level', 'job_duration_in_current_branch'],
#   ['job_duration_in_current_job_level', 'GPA'],
#   ['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_job_level', 'job_level_ordinary'],
#   ['job_duration_in_current_person_level', 'job_duration_in_current_branch'],
#   ['job_duration_in_current_person_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'number_of_dependences'],
#   ['job_duration_in_current_branch', 'job_level_ordinary'],
#   ['job_duration_in_current_branch', 'Education_level_ordinary'],
#   ['number_of_dependences', 'branch_rotation'],
#   ['GPA', 'Achievement_above_100_during3quartal']],
#  'divide': [['job_duration_in_current_job_level', 'year_graduated'],
#   ['job_duration_in_current_job_level', 'branch_rotation'],
#   ['job_duration_in_current_job_level', 'assign_of_otherposition'],
#   ['job_duration_in_current_job_level', 'Last_achievement'],
#   ['job_duration_in_current_job_level',
#    'Achievement_above_100_during3quartal'],
#   ['job_duration_in_current_job_level', 'Education_level_ordinary'],
#   ['job_duration_in_current_person_level', 'Last_achievement'],
#   ['job_duration_in_current_person_level',
#    'Achievement_above_100_during3quartal'],
#   ['job_duration_in_current_branch', 'job_rotation'],
#   ['job_duration_in_current_branch', 'assign_of_otherposition'],
#   ['job_duration_in_current_branch', 'Achievement_above_100_during3quartal'],
#   ['job_duration_in_current_branch', 'job_level_ordinary'],
#   ['gender', 'number_of_dependences'],
#   ['gender', 'branch_rotation'],
#   ['gender', 'Achievement_above_100_during3quartal'],
#   ['job_rotation', 'Last_achievement'],
#   ['job_rotation', 'Achievement_above_100_during3quartal'],
#   ['annual_leave', 'Last_achievement']],
#  'bin_numer_qcut': [['job_duration_in_current_branch', 5],
#   ['number_of_dependences', 5],
#   ['year_graduated', 10],
#   ['branch_rotation', 5],
#   ['Last_achievement', 10]],
#  'bin_numer_cut': [['job_duration_in_current_branch', 5],
#   ['job_duration_in_current_branch', 10],
#   ['number_of_dependences', 5],
#   ['branch_rotation', 5],
#   ['assign_of_otherposition', 5],
#   ['assign_of_otherposition', 10],
#   ['Last_achievement', 5]],
#  'bin_add_categ_numer_bin_cut': [['Education_level',
#    'job_duration_in_current_job_level',
#    10],
#   ['gender_str', 'job_duration_in_current_job_level', 10],
#   ['Achievement_above_100_during3quartal_str',
#    'job_duration_in_current_job_level',
#    10],
#   ['person_level', 'job_duration_in_current_branch', 10],
#   ['Employee_type', 'job_duration_in_current_branch', 10],
#   ['marital_status_maried', 'job_duration_in_current_branch', 10],
#   ['Education_level', 'job_duration_in_current_branch', 10],
#   ['gender_str', 'job_duration_in_current_branch', 10],
#   ['Achievement_above_100_during3quartal_str',
#    'job_duration_in_current_branch',
#    10],
#   ['person_level', 'gender', 10],
#   ['marital_status_maried', 'gender', 10],
#   ['marital_status_maried', 'number_of_dependences', 10],
#   ['Employee_type', 'branch_rotation', 10],
#   ['Achievement_above_100_during3quartal_str', 'branch_rotation', 10],
#   ['person_level', 'job_rotation', 10],
#   ['person_level', 'assign_of_otherposition', 10],
#   ['Achievement_above_100_during3quartal_str', 'assign_of_otherposition', 10],
#   ['Employee_type', 'annual_leave', 10],
#   ['gender_str', 'Last_achievement', 10],
#   ['Achievement_above_100_during3quartal_str', 'Last_achievement', 10]],
#  'bin_add_categ_numer_bin_qcut': [['Education_level',
#    'job_duration_in_current_job_level',
#    10],
#   ['person_level', 'job_duration_in_current_branch', 10],
#   ['Employee_type', 'job_duration_in_current_branch', 10],
#   ['marital_status_maried', 'job_duration_in_current_branch', 10],
#   ['gender_str', 'job_duration_in_current_branch', 10],
#   ['Achievement_above_100_during3quartal_str',
#    'job_duration_in_current_branch',
#    10],
#   ['person_level', 'gender', 10],
#   ['Education_level', 'GPA', 10],
#   ['Education_level', 'year_graduated', 10],
#   ['Employee_type', 'branch_rotation', 10],
#   ['marital_status_maried', 'job_rotation', 10],
#   ['gender_str', 'assign_of_otherposition', 10],
#   ['person_level', 'sick_leaves', 10]],
#  'bin_target_encoding_cut': [['job_duration_in_current_branch', 20],
#   ['number_of_dependences', 5],
#   ['branch_rotation', 5],
#   ['assign_of_otherposition', 10],
#   ['annual_leave', 5],
#   ['sick_leaves', 10],
#   ['Last_achievement', 5],
#   ['Last_achievement', 10],
#   ['Achievement_above_100_during3quartal', 5]],
#  'bin_target_encoding_qcut': [['job_duration_in_current_job_level', 5],
#   ['job_duration_in_current_job_level', 10],
#   ['job_duration_in_current_branch', 5],
#   ['number_of_dependences', 5],
#   ['assign_of_otherposition', 5],
#   ['sick_leaves', 5],
#   ['Last_achievement', 5],
#   ['Achievement_above_100_during3quartal', 5]],
#  'bin_target_encoding_custom_bin': [],
#  'categorical_mean_encoding': [],
#  'target': 'Best Performance'}

In [78]:
cv=2

# X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.5, random_state=3,shuffle=True)

add_fes,pipelines = fast_build_model_FE(X_train2,y_train2,cv,
                    Feature_Engineering,parameters_backup,model_base=LogisticRegression(solver='newton-cg',class_weight='balanced',random_state=0,max_iter=10000,C=0.01))

# add_fes,pipelines = fast_build_model_FE_1(X_train2,y_train2,
#                     Feature_Engineering,parameters_backup,model_base=LogisticRegression(solver='newton-cg',class_weight='balanced',random_state=0,max_iter=10000,C=0.01))



(2788, 42) (5576, 23)
(2788, 42) (5576, 23)
Fit iteration 0 done in : 0.5056495666503906
(2788, 42) (5576, 23)
(2788, 42) (5576, 23)
Fit iteration 1 done in : 0.45082664489746094
PRec Rec AUC average : [0.88002418 0.17820909] [0.60984293 0.50436954] <==> 0.5851936388415004
[0.5899748743718594, 0.5813085635304283]
0.5856417189511438


In [79]:
pred_proba = fast_predict_FE(X_test2.copy(),add_fes,pipelines)
roc_auc_score(y_test2.values, pred_proba,average='weighted')

# pred_proba = fast_predict_FE(X_train2.copy(),add_fes,pipelines)
# roc_auc_score(y_train2.values, pred_proba,average='weighted')

0.5665436097001723

In [82]:
# X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.5, random_state=3,shuffle=True)

# add_fes,pipelines = fast_build_model_FE(X_train2,y_train2,cv,
#                     Feature_Engineering,parameters_drop2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.01))


In [83]:
# pred_proba = fast_predict_FE(X_test2.copy(),add_fes,pipelines)

# roc_auc_score(y_test2.values, pred_proba,average='weighted')

In [23]:
cv=2
add_fes,pipelines = fast_build_model_FE(X_train,y_train,cv,
                    Feature_Engineering,parameters_backup,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.0001))

(5576, 171) (11153, 23)
(5577, 171) (11153, 23)
Fit iteration 0 done in : 5.409177780151367
(5577, 171) (11153, 23)
(5576, 171) (11153, 23)
Fit iteration 1 done in : 5.338063716888428
PRec Rec AUC average : [0.87511931 0.1750565 ] [0.57807902 0.52046426] <==> 0.5604599834173124
[0.565341272150856, 0.5567539832540433]
0.5610476277024496


In [13]:
cv=5
add_fes,pipelines = fast_build_model_FE(X_train,y_train,cv,
                    Feature_Engineering,parameters_backup,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.0001))

(8922, 171) (11153, 23)
(2231, 171) (11153, 23)
Fit iteration 0 done in : 7.69919228553772
(8922, 171) (11153, 23)
(2231, 171) (11153, 23)
Fit iteration 1 done in : 7.944198131561279
(8922, 171) (11153, 23)
(2231, 171) (11153, 23)
Fit iteration 2 done in : 7.729879856109619
(8923, 171) (11153, 23)
(2230, 171) (11153, 23)
Fit iteration 3 done in : 8.474101543426514
(8923, 171) (11153, 23)
(2230, 171) (11153, 23)
Fit iteration 4 done in : 7.472262620925903
PRec Rec AUC average : [0.8763625  0.17818643] [0.59142497 0.5149664 ] <==> 0.5753824764284722
[0.5991635186184565, 0.564666829011958, 0.5918286915396742, 0.5916410753341336, 0.5350171385595897]
0.5819151738152073


In [16]:
cv=10
add_fes,pipelines = fast_build_model_FE(X_train,y_train,cv,
                    Feature_Engineering,parameters_backup,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.0001))

(10037, 171) (11153, 23)
(1116, 171) (11153, 23)
Fit iteration 0 done in : 7.970711708068848
(10037, 171) (11153, 23)
(1116, 171) (11153, 23)
Fit iteration 1 done in : 8.354789972305298
(10037, 171) (11153, 23)
(1116, 171) (11153, 23)
Fit iteration 2 done in : 10.407728910446167
(10038, 171) (11153, 23)
(1115, 171) (11153, 23)
Fit iteration 3 done in : 8.743772506713867
(10038, 171) (11153, 23)
(1115, 171) (11153, 23)
Fit iteration 4 done in : 7.586367130279541
(10038, 171) (11153, 23)
(1115, 171) (11153, 23)
Fit iteration 5 done in : 7.7707109451293945
(10038, 171) (11153, 23)
(1115, 171) (11153, 23)
Fit iteration 6 done in : 8.127434730529785
(10038, 171) (11153, 23)
(1115, 171) (11153, 23)
Fit iteration 7 done in : 7.865878343582153
(10038, 171) (11153, 23)
(1115, 171) (11153, 23)
Fit iteration 8 done in : 7.926168441772461
(10038, 171) (11153, 23)
(1115, 171) (11153, 23)
Fit iteration 9 done in : 7.732975959777832
PRec Rec AUC average : [0.87778456 0.18177865] [0.60456074 0.5106902

In [91]:
data_skf = {'train':[],'val':[]}
cv =5
skf = StratifiedKFold(n_splits=cv,random_state = 3,shuffle = True)



for train_index,val_index in skf.split(X_train,y_train):
    add_fe = Feature_Engineering(parameters_backup)
    add_fe.fit(data.iloc[train_index,:])
    data_skf['train'].append([add_fe.transform(X_train.iloc[train_index,:],mode='val'),y_train.iloc[train_index]])
    data_skf['val'].append([add_fe.transform(X_train.iloc[val_index,:],mode='val'),y_train.iloc[val_index]])
    break
num_cols_fe = list(data_skf['train'][0][0].select_dtypes(exclude='object').columns)
cat_cols_fe = list(data_skf['train'][0][0].select_dtypes(include='object').columns)
len(num_cols_fe),len(cat_cols_fe)

(19, 9)

In [92]:
import copy
parameters_fe = {'multiply':[],
              'add':[],
              'add_str':[],
              'substract':[],'divide':[],
              'bin_numer_qcut':[],
              'bin_numer_cut':[],
              'bin_add_categ_numer_bin_cut':[],
              'bin_add_categ_numer_bin_qcut':[],
            'bin_target_encoding_cut':[],
             'bin_target_encoding_qcut':[],
             'bin_target_encoding_custom_bin':[],
              'categorical_mean_encoding':[],
             'target':'Best Performance'}

cv=2
num_cols_2 = copy.deepcopy(num_cols_fe)  # 16
cat_cols_2 = copy.deepcopy(cat_cols_fe)   # 6

# auc_end = 0
auc_end = 0.5903428283077937
# auc_end = 0.5950855235807718
for i,param in enumerate(parameters_fe):
    if i in [0,1,3,4]:
        print(param)
        for j in range(len(num_cols_2)):
            if j!=len(num_cols_2)-1:
                for k in range(len(num_cols_2[j+1:])):
                    if k>=15:
                        parameters_fe2 = copy.deepcopy(parameters_fe)
                        col_col = [num_cols_2[j],num_cols_2[j+1+k]]
                        parameters_fe2[param].append(col_col)
        #                 print(parameters_fe)
        #                 print(parameters_fe2)
                        

                        try:
                            auc = fast_build_model_FE_automatic_2(X_train2,y_train2,cv,
                                            Feature_Engineering,parameters_backup,parameters_fe2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.1))
                        except ZeroDivisionError:
                            pass
                        if auc>auc_end+0.0005:
                            auc_end = auc.copy()
                            parameters_fe[param].append(col_col)
                            print(parameters_fe[param][-1],auc)
                        else:
        #                     print(parameters_fe)
                            continue
    elif i in [2]:
        print(param)
        for j in range(len(cat_cols_2)):
            if j!=len(cat_cols_2)-1:
                for k in range(len(cat_cols_2[j+1:])):
                    if k>=5:
                        parameters_fe2 = copy.deepcopy(parameters_fe)
                        col_col = [cat_cols_2[j],cat_cols_2[j+1+k]]
                        parameters_fe2[param].append(col_col)
        #                 print(parameters_fe)
        #                 print(parameters_fe2)
        #                 print()
                        

                        try:
                            auc = fast_build_model_FE_automatic_2(X_train2,y_train2,cv,
                                            Feature_Engineering,parameters_backup,parameters_fe2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.1))
                        except ZeroDivisionError:
                            pass
                        if auc>auc_end+0.0005:
                            auc_end = auc.copy()
                            parameters_fe[param].append(col_col)
                            print(parameters_fe[param][-1],auc)
                        else:
        #                     print(parameters_fe)
                            continue
    elif i in [5,6,9,10]:
        print(param)
        for j in range(len(num_cols_2)):
            if j!=len(num_cols_2)-1 and j>15:
                bins = [5,10,20]
                for bin_ in bins:
                    parameters_fe2 = copy.deepcopy(parameters_fe)
                    col_col = [num_cols_2[j],bin_]
                    parameters_fe2[param].append(col_col)
    #                 print(parameters_fe)
    #                 print(parameters_fe2)
    #                 print()
                    
                    
                    try:
                        auc = fast_build_model_FE_automatic_2(X_train2,y_train2,cv,
                                        Feature_Engineering,parameters_backup,parameters_fe2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.1))
                    except ZeroDivisionError:
                        pass
                    if auc>auc_end+0.0005:
                        auc_end = auc.copy()
                        parameters_fe[param].append(col_col)
                        print(parameters_fe[param][-1],auc)
                    else:
    #                     print(parameters_fe)
                        continue
        
    elif i in [7,8]:
        print(param)
        for j in range(len(num_cols_2)):
            if j!=len(num_cols_2)-1:
                bins = [10]
                for bin_ in bins:
                    for k,col2 in enumerate(cat_cols_2):
                        if k>=6:
                            parameters_fe2 = copy.deepcopy(parameters_fe)
                            col_col = [col2,num_cols_2[j],bin_]
                            parameters_fe2[param].append(col_col)
            #                 print(parameters_fe)
            #                 print(parameters_fe2)
            #                 print()
                            try:
                                auc = fast_build_model_FE_automatic_2(X_train2,y_train2,cv,
                                                Feature_Engineering,parameters_backup,parameters_fe2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.1))
                            except ZeroDivisionError:
                                pass
                            if auc>auc_end+0.0005:
                                auc_end = auc.copy()
                                parameters_fe[param].append(col_col)
                                print(parameters_fe[param][-1],auc)
                            else:
            #                     print(parameters_fe)
                                continue
        
    elif i in [12]:
        print(param)
        for j in range(len(cat_cols_2)):
            if j!=len(cat_cols_2)-1:
                if j>=6:
                    parameters_fe2 = copy.deepcopy(parameters_fe)
                    col_col = cat_cols_2[j]
                    parameters_fe2[param].append(col_col)
    #                 print(parameters_fe)
    #                 print(parameters_fe2)
    #                 print()

#                     print(col_col)
                    try:
                        auc = fast_build_model_FE_automatic_2(X_train2,y_train2,cv,
                                        Feature_Engineering,parameters_backup,parameters_fe2,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=10000,C=0.1))
                    except ZeroDivisionError:
                        pass
                    if auc>auc_end+0.0005:
                        auc_end = auc.copy()
                        parameters_fe[param].append(col_col)
                        print(parameters_fe[param][-1],auc)
                    else:
    #                     print(parameters_fe)
                        continue
    print('=========================\n')



multiply
['job_duration_in_current_job_level', 'job_duration_in_current_job_level_times_job_duration_in_current_person_level'] 0.5906770060965563
['job_duration_in_current_person_level', 'job_duration_in_current_job_level_times_job_duration_in_current_person_level'] 0.5908116555445708
['job_duration_in_current_branch', 'job_duration_in_current_job_level_divide_branch_rotation'] 0.5909995983687593

add
['job_duration_in_current_job_level', 'job_duration_in_current_job_level_times_job_duration_in_current_person_level'] 0.5910907377656945
['job_duration_in_current_job_level', 'job_duration_in_current_job_level_divide_branch_rotation'] 0.5911177706376668
['job_duration_in_current_person_level', 'job_duration_in_current_job_level_times_job_duration_in_current_person_level'] 0.5911836793540947

add_str


KeyboardInterrupt: 

mix ordinary and fe features not yet

In [72]:
parameters_fe

{'multiply': [['job_duration_in_current_job_level_bin_target_encoding_cut',
   'job_duration_in_current_person_level_bin_target_encoding_cut'],
  ['job_duration_in_current_person_level_bin_target_encoding_cut',
   'job_duration_in_current_branch_bin_target_encoding_cut'],
  ['job_duration_in_current_branch_bin_target_encoding_cut',
   'gender_bin_target_encoding_cut'],
  ['number_of_dependences_bin_target_encoding_cut',
   'GPA_bin_target_encoding_cut'],
  ['job_rotation_bin_target_encoding_cut',
   'assign_of_otherposition_bin_target_encoding_cut'],
  ['assign_of_otherposition_bin_target_encoding_cut',
   'annual_leave_bin_target_encoding_cut'],
  ['Achievement_above_100_during3quartal_bin_target_encoding_cut',
   'job_level_ordinary_bin_target_encoding_cut'],
  ['assign_of_otherposition_bin_target_encoding_qcut',
   'marital_status_maried_categorical_mean_encoding'],
  ['Last_achievement_times_Achievement_above_100_during3quartal',
   'job_duration_in_current_branch_plus_gender'],
  

In [104]:
parameters_fe = {'multiply': [['job_duration_in_current_job_level_bin_target_encoding_cut',
   'job_duration_in_current_person_level_bin_target_encoding_cut'],
  ['job_duration_in_current_person_level_bin_target_encoding_cut',
   'job_duration_in_current_branch_bin_target_encoding_cut'],
  ['job_duration_in_current_branch_bin_target_encoding_cut',
   'gender_bin_target_encoding_cut'],
  ['number_of_dependences_bin_target_encoding_cut',
   'GPA_bin_target_encoding_cut'],
  ['job_rotation_bin_target_encoding_cut',
   'assign_of_otherposition_bin_target_encoding_cut'],
  ['assign_of_otherposition_bin_target_encoding_cut',
   'annual_leave_bin_target_encoding_cut'],
  ['Achievement_above_100_during3quartal_bin_target_encoding_cut',
   'job_level_ordinary_bin_target_encoding_cut'],
  ['assign_of_otherposition_bin_target_encoding_qcut',
   'marital_status_maried_categorical_mean_encoding'],
  ['Last_achievement_times_Achievement_above_100_during3quartal',
   'job_duration_in_current_branch_plus_gender'],
  ['gender_plus_number_of_dependences',
   'assign_of_otherposition_plus_annual_leave'],
  ['job_duration_in_current_branch_minus_gender', 'GPA_minus_year_graduated']],
 'add': [],
 'add_str': [['Last_achievement_bin_numer_cut',
   'Achievement_above_100_during3quartal_bin_numer_cut'],
  ['marital_status_maried_Achievement_above_100_during3quartal_bin_add_categ_numer_bin_qcut',
   'gender_str_Achievement_above_100_during3quartal_bin_add_categ_numer_bin_qcut']],
 'substract': [['job_duration_in_current_branch_bin_target_encoding_cut',
   'gender_bin_target_encoding_cut'],
  ['marital_status_maried_categorical_mean_encoding',
   'job_duration_in_current_job_level_times_job_duration_in_current_person_level'],
  ['job_duration_in_current_job_level_times_job_duration_in_current_person_level',
   'number_of_dependences_times_GPA']],
 'divide': [['GPA_bin_target_encoding_cut',
   'job_rotation_bin_target_encoding_cut'],
  ['job_rotation_bin_target_encoding_cut',
   'assign_of_otherposition_bin_target_encoding_cut'],
  ['annual_leave_bin_target_encoding_cut',
   'Achievement_above_100_during3quartal_bin_target_encoding_cut'],
  ['job_level_ordinary_bin_target_encoding_cut',
   'year_graduated_bin_target_encoding_qcut'],
  ['job_duration_in_current_branch_minus_gender', 'GPA_minus_year_graduated'],
  ['job_rotation_divide_assign_of_otherposition',
   'sick_leaves_divide_Last_achievement']],
 'bin_numer_qcut': [['job_duration_in_current_person_level_bin_target_encoding_cut',
   5],
  ['gender_bin_target_encoding_cut', 5],
  ['assign_of_otherposition_bin_target_encoding_cut', 5],
  ['job_level_ordinary_bin_target_encoding_cut', 5],
  ['job_duration_in_current_person_level_bin_target_encoding_cut', 5],
  ['assign_of_otherposition_plus_annual_leave', 5],
  ['GPA_minus_year_graduated', 5],
  ['job_rotation_divide_assign_of_otherposition', 10]],
 'bin_numer_cut': [['Achievement_above_100_during3quartal_bin_target_encoding_cut',
   10],
  ['Achievement_above_100_during3quartal_bin_target_encoding_cut', 20],
  ['year_graduated_bin_target_encoding_qcut', 5],
  ['year_graduated_bin_target_encoding_qcut', 10],
  ['year_graduated_bin_target_encoding_qcut', 20],
  ['assign_of_otherposition_bin_target_encoding_qcut', 5],
  ['number_of_dependences_times_GPA', 5],
  ['assign_of_otherposition_times_annual_leave', 5],
  ['Last_achievement_times_Achievement_above_100_during3quartal', 5],
  ['Last_achievement_plus_Achievement_above_100_during3quartal', 20],
  ['job_duration_in_current_job_level_minus_job_duration_in_current_person_level',
   5],
  ['GPA_minus_year_graduated', 5],
  ['year_graduated_minus_branch_rotation', 5]],
 'bin_add_categ_numer_bin_cut': [['job_duration_in_current_job_level_bin_numer_qcut',
   'job_duration_in_current_job_level_bin_target_encoding_cut',
   10],
  ['GPA_bin_numer_cut',
   'job_duration_in_current_job_level_bin_target_encoding_cut',
   10],
  ['gender_str_job_duration_in_current_job_level_bin_add_categ_numer_bin_cut',
   'job_duration_in_current_job_level_bin_target_encoding_cut',
   10]],
 'bin_add_categ_numer_bin_qcut': [],
 'bin_target_encoding_cut': [],
 'bin_target_encoding_qcut': [],
 'bin_target_encoding_custom_bin': [],
 'categorical_mean_encoding': [],
 'target': 'Best Performance'}




In [92]:
# X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.5, random_state=3,shuffle=True)


In [185]:
# cv=5
# add_fes1,add_fes2,pipelines = fast_build_model_FE_automatic_2_pipe(X_train2,y_train2,cv,
#                     Feature_Engineering,parameters_backup,parameters_fe,model_base=MLPClassifier(hidden_layer_sizes=(50,),
#                                                                                                 activation='identity',
#                                                                                                 alpha=0.001,
#                                                                                                 random_state=0,
#                                                                                                  max_iter=20000))

In [188]:
cv=5
add_fes1,add_fes2,pipelines = fast_build_model_FE_automatic_2_pipe(X_train2,y_train2,cv,
                    Feature_Engineering,parameters_backup,parameters_fe,model_base=LogisticRegression(solver='newton-cg',class_weight='balanced',random_state=0,max_iter=20000,C=0.01))

Fit iteration 0 done in : 8.156850814819336
Fit iteration 1 done in : 5.634549856185913
Fit iteration 2 done in : 5.0862181186676025
Fit iteration 3 done in : 5.7015910148620605
Fit iteration 4 done in : 5.564695358276367
PRec Rec AUC average : [0.87331378 0.17728532] [0.62563025 0.47058824] <==> 0.5778979238754325
0.5795683689456629


In [172]:
# cv=2
# add_fes1,add_fes2,pipelines = fast_build_model_FE_automatic_2_pipe(X_train2,y_train2,cv,
#                     Feature_Engineering,parameters_backup,parameters_fe,model_base=SVC(class_weight='balanced',probability=True,kernel='linear',gamma='auto',random_state=0,C=0.01))

In [189]:
pred_proba = fast_predict_FE_2(X_test2.copy(),add_fes1,add_fes2,pipelines)

roc_auc_score(y_test2.values, pred_proba,average='weighted')

0.5644752599191328

In [163]:
# pred_proba2 = fast_predict_FE(X_test2.copy(),add_fes,pipelines)
pred_proba2 = fast_predict_FE_2(X_test2.copy(),add_fes1,add_fes2,pipelines)

roc_auc_score(y_test2.values, (pred_proba+pred_proba2)/2,average='weighted')

0.5765182565723763

In [21]:
X_ha = data_test.copy()
add_fes2[1].transform(add_fes1[1].transform(X_ha,mode='test'),mode='test')

,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,gender,marital_status_maried,number_of_dependences,Education_level,GPA,...,marital_status_maried_Achievement_above_100_during3quartal_bin_add_categ_numer_bin_qcut_plus_gender_str_Achievement_above_100_during3quartal_bin_add_categ_numer_bin_qcut,job_duration_in_current_branch_bin_target_encoding_cut_minus_gender_bin_target_encoding_cut,marital_status_maried_categorical_mean_encoding_minus_job_duration_in_current_job_level_times_job_duration_in_current_person_level,job_duration_in_current_job_level_times_job_duration_in_current_person_level_minus_number_of_dependences_times_GPA,GPA_bin_target_encoding_cut_divide_job_rotation_bin_target_encoding_cut,job_rotation_bin_target_encoding_cut_divide_assign_of_otherposition_bin_target_encoding_cut,annual_leave_bin_target_encoding_cut_divide_Achievement_above_100_during3quartal_bin_target_encoding_cut,job_level_ordinary_bin_target_encoding_cut_divide_year_graduated_bin_target_encoding_qcut,job_duration_in_current_branch_minus_gender_divide_GPA_minus_year_graduated,job_rotation_divide_assign_of_otherposition_divide_sick_leaves_divide_Last_achievement
0,1.256981,PG03,1.256981,0.707107,RM_type_A,1,N,0,level_3,3.08,...,"N_(-0.001, inf]_1_(-0.001, inf]",-0.037096,-1.416596,1.58,1.005129,1.242308,0.982086,0.980657,0.000146,4.000952e+00
1,1.224745,PG03,1.224745,1.256981,RM_type_B,2,N,0,level_4,3.31,...,"N_(-0.001, inf]_2_(-0.001, inf]",0.026256,-1.336596,1.50,1.005129,0.983905,0.982086,0.908693,0.000370,2.000000e+08
2,0.500000,PG03,0.500000,1.081665,RM_type_A,2,Y,1,level_4,3.44,...,"Y_(-0.001, inf]_2_(-0.001, inf]",0.125542,-0.106202,-3.19,1.005129,0.983905,1.217694,0.984076,0.000457,1.763800e+06
3,1.256981,PG03,1.256981,1.802776,RM_type_A,2,Y,1,level_4,0.00,...,"Y_(-0.001, inf]_2_(-0.001, inf]",0.003959,-1.436202,1.58,1.005129,0.983905,0.982086,0.908693,0.000098,3.000000e+08
4,1.352775,PG03,1.352775,1.224745,RM_type_B,1,Y,0,level_4,3.34,...,"Y_(-0.001, inf]_1_(-0.001, inf]",0.013061,-1.686202,1.83,1.005129,0.983905,0.982086,1.054285,-0.000112,2.000000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1.292285,PG03,1.292285,0.707107,RM_type_B,2,Y,0,level_4,3.01,...,"Y_(-0.001, inf]_2_(-0.001, inf]",-0.023901,-1.526202,1.67,1.005129,0.983905,0.982086,0.908693,0.000643,2.000000e+08
5996,1.352775,PG03,1.352775,1.581139,RM_type_A,2,Y,1,level_4,3.56,...,"Y_(-0.001, inf]_2_(-0.001, inf]",-0.023351,-1.686202,-1.73,1.005129,1.242308,0.967674,0.984076,0.000209,2.857143e+03
5997,1.385641,PG03,1.385641,1.224745,RM_type_A,2,Y,0,level_4,3.59,...,"Y_(-0.001, inf]_2_(-0.001, inf]",0.026256,-1.776202,1.92,1.005129,0.983905,1.211128,0.908693,0.000386,3.000000e+08
5998,1.385641,PG05,1.385641,0.648074,RM_type_A,2,Y,2,level_4,2.93,...,"Y_(-0.001, inf]_2_(-0.001, inf]",-0.008831,-1.776202,-3.94,1.037787,1.203213,0.982086,1.080526,0.000676,7.500000e+03


In [194]:
cv=5
add_fes1,add_fes2,pipelines = fast_build_model_FE_automatic_2_pipe(X_train,y_train,cv,
                    Feature_Engineering,parameters_backup,parameters_fe,model_base=LogisticRegression(solver='newton-cg',class_weight='balanced',random_state=0,max_iter=20000,C=0.01))

Fit iteration 0 done in : 7.88448429107666
Fit iteration 1 done in : 7.247299909591675
Fit iteration 2 done in : 7.898740291595459
Fit iteration 3 done in : 7.478065252304077
Fit iteration 4 done in : 7.358240842819214
PRec Rec AUC average : [0.87791901 0.18486437] [0.62421185 0.49541845] <==> 0.5909752869680567
0.5911841992015716


In [195]:
pred_proba = fast_predict_FE_2(data_test.copy(),add_fes1,add_fes2,pipelines)

df_submission = pd.DataFrame({'index':data_test.index,'Best Performance':pred_proba})
df_submission

df_submission.to_csv('df_submission_22feb_LOGREG5CV059C01_FE.csv',index=False)

In [22]:
# cv=20
# add_fes1,add_fes2,pipelines = fast_build_model_FE_automatic_2_pipe(X_train,y_train,cv,
#                     Feature_Engineering,parameters_backup,parameters_fe,model_base=LogisticRegression(class_weight='balanced',random_state=0,max_iter=20000))

Fit iteration 0 done in : 13.59222674369812
Fit iteration 1 done in : 11.736554861068726
Fit iteration 2 done in : 19.517606258392334
Fit iteration 3 done in : 14.122777223587036
Fit iteration 4 done in : 18.369574069976807
Fit iteration 5 done in : 21.2182834148407
Fit iteration 6 done in : 18.07462978363037
Fit iteration 7 done in : 16.51407289505005
Fit iteration 8 done in : 17.289663076400757
Fit iteration 9 done in : 15.852121114730835
Fit iteration 10 done in : 18.008126497268677
Fit iteration 11 done in : 15.563772916793823
Fit iteration 12 done in : 11.28114914894104
Fit iteration 13 done in : 16.022382259368896
Fit iteration 14 done in : 22.8385066986084
Fit iteration 15 done in : 11.751692533493042
Fit iteration 16 done in : 23.057936668395996
Fit iteration 17 done in : 21.86928963661194
Fit iteration 18 done in : 11.695711135864258
Fit iteration 19 done in : 14.633818864822388
PRec Rec AUC average : [0.88384592 0.19281706] [0.62211013 0.52474038] <==> 0.5986764919989431
0.59

In [86]:
# pred_proba = fast_predict_FE_2(data_test.copy(),add_fes1,add_fes2,pipelines)

# df_submission = pd.DataFrame({'index':data_test.index,'Best Performance':pred_proba})
# df_submission

# df_submission.to_csv('df_submission_21feb_LOGREG10CV0598_FE.csv',index=False)

# ensemble

In [269]:
class Ensemble_Models():
    def __init__(self,list_models):
        self.list_models = list_models
        
    def fit(self,X,y):
        
        num_transformer = Pipeline(steps=[
                                    ('imputer', SimpleImputer(strategy = 'median')),
                                    ('scaler', RobustScaler())
                                    ])

        cat_transformer = Pipeline(steps=[
                                        ('imputer', SimpleImputer(strategy='most_frequent')),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                        ])
        num_cols_fe = list(X.select_dtypes(exclude='object').columns)
        cat_cols_fe = list(X.select_dtypes(include='object').columns)
        
        
        transformer = ColumnTransformer(
            transformers=[
                ('num', num_transformer, num_cols_fe),
                ('cat', cat_transformer, cat_cols_fe)
            ])
        pipelines=[]
        for mod in self.list_models:
            main_pipeline = Pipeline(steps=[('transformer', transformer),
                          ('classifier', mod)])
            model = clone(main_pipeline)
            model.fit(X,y)
            pipelines.append(model)
        
        self.pipelines = pipelines
        
    def predict_proba(self,X):
        predict_proba = np.zeros(len(X))
        for i,pipe in enumerate(self.pipelines):
            predict_proba += pipe.predict_proba(X)[:,1] / len(self.pipelines)
            
        return predict_proba
    
    def predict(self,X,threshold=0.5):
        predict_proba = np.zeros(len(X))
        for i,pipe in enumerate(self.pipelines):
            predict_proba += pipe.predict_proba(X)[:,1] / len(self.pipelines)
        
        predict = np.where(predict_proba>threshold,1,0)
        
        return predict
            
            
        

In [270]:
def fast_build_model_FE_ensemble(X,y,cv,Feature_Engineering,parameters,list_models=[]):

    
    skf = StratifiedKFold(n_splits=cv,random_state = 0)

    # oof validation
    oof_y_valid = []
    oof_y_valid_pred = []
    oof_y_valid_pred_proba = []
    pipelines = []
    add_fes = []
    data = pd.concat([X,y],axis=1)
    aucs = []
    
#     print(data.columns)
    for cv,(train_index, val_index) in enumerate(skf.split(X,y)):
        start_fit = time.time()
        data_train = data.iloc[train_index,:].copy()
#         data_val = data.iloc[val_index,:][features]
        
        add_fe = Feature_Engineering(parameters)
        add_fe.fit(data_train)
        
        X_train = add_fe.transform(data_train).drop(columns=[parameters['target']])
#         X_train = data_train.drop(columns=[parameters['target']])
    
        print(X_train.shape,data.shape)
        y_train = y.iloc[train_index]
        
        X_val = add_fe.transform(X.iloc[val_index,:],mode='val')
#         X_val = X.iloc[val_index,:]
    
        y_val = y.iloc[val_index]
        print(X_val.shape,data.shape)
        
        model = Ensemble_Models(list_models)

        
        add_fes.append(add_fe)
        model.fit(X_train,y_train.values.ravel())
        pred = model.predict(X_val)
        pred_proba = model.predict_proba(X_val)
        
        oof_y_valid_pred.extend(pred)
        oof_y_valid_pred_proba.extend(pred_proba)
        oof_y_valid.extend(y_val.values)

        pipelines.append(model)
        
        
        
        aucs.append(roc_auc_score(y_val.values, pred_proba,average='micro'))
        
        
        print(f'Fit iteration {cv} done in : {str(time.time()-start_fit)}')

    prec,rec,f1, _ = precision_recall_fscore_support(oof_y_valid,oof_y_valid_pred)
    auc = roc_auc_score(oof_y_valid, oof_y_valid_pred_proba,average='micro')
    print(f'PRec Rec AUC average : {prec} {rec} <==> {auc}')
    print(aucs)
    print(np.mean(aucs[:2]))
    
    return add_fes,pipelines


def fast_predict_FE_ensemble(X,add_fes,pipelines):
    data = X.copy()
#     pred = np.zeros(1,len(X))
    pred_proba = np.zeros((len(X)))
    for i in range(len(pipelines)):
        
        pred_proba += pipelines[i].predict_proba(add_fes[i].transform(data,mode='test')) / len(pipelines)
    
    return pred_proba
        

In [283]:
list_models = [
#     LogisticRegression(class_weight='balanced',random_state = 0),
    XGBClassifier(scale_pos_weight=3,random_state = 0),
    LGBMClassifier(scale_pos_weight=3,random_state = 0),
    RandomForestClassifier(class_weight='balanced',random_state = 0),
#     MLPClassifier(hidden_layer_sizes=(100,),activation='identity',random_state = 0)
]



In [284]:
parameters = {'multiply':[['GPA','number_of_dependences']],
              'add':[['annual_leave','sick_leaves'],['assign_of_otherposition','branch_rotation']],
              'add_str':[['Education_level','job_level']],
              'substract':[],'divide':[],
              'bin_numer_qcut':[],
              'bin_numer_cut':[],
              'bin_add_categ_numer_bin_qcut':[['job_level','GPA',5],['Education_level','GPA',5]],
            'bin_target_encoding_cut':[],
             'bin_target_encoding_qcut':[['year_graduated',5],['GPA',5],['annual_leave',5]],
             'bin_target_encoding_custom_bin':[],
              'categorical_mean_encoding':['job_level','person_level','Employee_type','Education_level'],
             'target':'Best Performance'}

# parameters = {'multiply':[],
#               'add':[],
#               'add_str':[],
#               'substract':[],'divide':[],
#               'bin_numer_qcut':[],
#               'bin_numer_cut':[],
#               'bin_add_categ_numer_bin_qcut':[],
#             'bin_target_encoding_cut':[['GPA',30]],
#              'bin_target_encoding_qcut':[],
#              'bin_target_encoding_custom_bin':[],
#               'categorical_mean_encoding':[],
#              'target':'Best Performance'}

In [285]:
cv=10
add_fes_str,pipelines_str = fast_build_model_FE_ensemble(X_train,y_train,cv,
                    Feature_Engineering,parameters,list_models=list_models)

E:\Anaconda3\envs\andre\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



GPA
GPA
year_graduated
GPA
annual_leave
(9033, 34) (10037, 22)
(1004, 34) (10037, 22)


<ipython-input-18-3e64f3dfca2c>:124: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:138: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3

[14:49:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 0 done in : 3.4611732959747314
GPA
GPA
year_graduated
GPA
annual_leave
(9033, 34) (10037, 22)
(1004, 34) (10037, 22)


<ipython-input-18-3e64f3dfca2c>:124: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:138: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3

...

[14:50:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 7 done in : 4.467153310775757
GPA
GPA
year_graduated
GPA
annual_leave
(9034, 34) (10037, 22)
(1003, 34) (10037, 22)


<ipython-input-18-3e64f3dfca2c>:124: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:138: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3

[14:50:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 8 done in : 4.683999061584473
GPA
GPA
year_graduated
GPA
annual_leave
(9034, 34) (10037, 22)
(1003, 34) (10037, 22)


<ipython-input-18-3e64f3dfca2c>:124: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3e64f3dfca2c>:138: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-3

[14:50:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 9 done in : 5.395112752914429
PRec Rec AUC average : [0.85593476 0.25961538] [0.99102459 0.01851852] <==> 0.5718078774357457
[0.5900708880160935, 0.5850416706581091, 0.5772583580802758, 0.6289555193664783, 0.5740332726634096, 0.5449436408340518, 0.5569259507615671, 0.533936178763765, 0.5465798569246845, 0.579634276945701]
0.5875562793371013


In [286]:
pred_proba = fast_predict_FE_ensemble(X_val,add_fes_str,pipelines_str)
roc_auc_score(y_val.values, pred_proba,average='micro')

E:\Anaconda3\envs\andre\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning:

Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.

E:\Anaconda3\envs\andre\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning:

Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.

E:\Anaconda3\envs\andre\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning:

Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.

E:\Anaconda3\envs\andre\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning:

Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.

E:\Anaconda3\envs\andre\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning:

Given feature/colu

0.5762417796008895

In [136]:
pred_proba = fast_predict_FE_ensemble(data_test,add_fes_str,pipelines_str)

# df_submission = pd.DataFrame({'index':data_test.index,'Best Performance':pred_proba})
# df_submission

# df_submission.to_csv('df_submission_13feb_ENSEMBLESTR5CV_FE.csv',index=False)

In [138]:
(pred_proba_ENSE==pred_proba_XGB).all()

False

In [148]:
(pred_proba_ENSE.round(4)==pred_proba_XGB.round(4)).all()

True

In [145]:
pred_proba_XGB[0]

0.17933059739880264

In [146]:
pred_proba_ENSE[0]

0.17933059819042685

In [192]:
cv=5
add_fes,pipelines = fast_build_model_FE_ensemble(X_train,y_train,cv,
                    Feature_Engineering,parameters,list_models=list_models)

E:\Anaconda3\envs\andre\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
<ipython-input-19-3e64f3dfca2c>:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{col}_bin_target_encoding_cut'] = pd.cut(data[col],bins=bin_dummy)
<ipython-input-19-3e64f3dfca2c>:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

(8922, 22) (11153, 22)
(2231, 22) (11153, 22)


E:\Anaconda3\envs\andre\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\Anaconda3\envs\andre\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:11:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 0 done in : 3.1989974975585938
(8922, 22) (11153, 22)
(2231, 22) (11153, 22)


<ipython-input-19-3e64f3dfca2c>:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{col}_bin_target_encoding_cut'] = pd.cut(data[col],bins=bin_dummy)
<ipython-input-19-3e64f3dfca2c>:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{col}_bin_target_encoding_cut'] = pd.merge(data,data_dummy,how='left',on=[f'{col}_bin_target_encoding_cut'])[f'{target_encode}'].values
E:\Anaconda3\envs\andre\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (st

...

[22:11:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 2 done in : 3.7990024089813232
(8923, 22) (11153, 22)
(2230, 22) (11153, 22)


<ipython-input-19-3e64f3dfca2c>:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{col}_bin_target_encoding_cut'] = pd.cut(data[col],bins=bin_dummy)
<ipython-input-19-3e64f3dfca2c>:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{col}_bin_target_encoding_cut'] = pd.merge(data,data_dummy,how='left',on=[f'{col}_bin_target_encoding_cut'])[f'{target_encode}'].values
E:\Anaconda3\envs\andre\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (st

[22:11:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 3 done in : 3.40883731842041
(8923, 22) (11153, 22)
(2230, 22) (11153, 22)


<ipython-input-19-3e64f3dfca2c>:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{col}_bin_target_encoding_cut'] = pd.cut(data[col],bins=bin_dummy)
<ipython-input-19-3e64f3dfca2c>:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{col}_bin_target_encoding_cut'] = pd.merge(data,data_dummy,how='left',on=[f'{col}_bin_target_encoding_cut'])[f'{target_encode}'].values
E:\Anaconda3\envs\andre\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (st

[22:11:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fit iteration 4 done in : 3.3530004024505615
PRec Rec AUC average : [0.85363661 0.3       ] [0.99779319 0.00549786] <==> 0.5593385721068307
[0.5534365122195667, 0.574695922997065, 0.5622765082091179, 0.5345414692076409, 0.5722752261438161]
0.5640662176083158


0.5548748462799754

In [117]:
# import pickle
# with open("pipelines_0567_ensembleFE.pkl", "wb") as f:
#     pickle.dump(pipelines, f)

In [121]:
# pipelines[0].pipelines[0]['transformer'].transform(data2_fe.drop(columns=['Best Performance']).copy()).shape

In [122]:
pipelines[0].pipelines[0]['transformer']

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['job_duration_in_current_job_level',
                                  'job_duration_in_current_person_level',
                                  'job_duration_in_current_branch', 'age',
                                  'number_of_dependences', 'GPA',
                                  'year_graduated',
                                  'job_duration_from_training',
                                  'branch_rotation', 'job_rotation',
                                  'assign_of_otherposition', 'annual_leave',
                                  'sick_leaves', 'Last_achievement',
                                  'Achievement_above_100%_during3quartal',
                                  'GPA_bin_

In [123]:
data2_fe.shape

(11153, 36)

In [124]:
# cv=10
# add_fes,pipelines = fast_build_model_FE_ensemble(X_train,y_train,cv,
#                     Feature_Engineering,parameters,list_models=list_models)

In [125]:
len(add_fes)

5

In [126]:
pred_proba = fast_predict_FE_ensemble(data_test,add_fes,pipelines)

In [127]:
df_submission = pd.DataFrame({'index':data_test.index,'Best Performance':pred_proba})
df_submission

,index,Best Performance
0,0,0.230511
1,1,0.273958
2,2,0.386696
3,3,0.220155
4,4,0.253016
...,...,...
5995,5995,0.240473
5996,5996,0.168555
5997,5997,0.247632
5998,5998,0.196402


In [45]:
# df_submission.to_csv('df_submission_12feb_Ensemble_FE.csv',index=False)

In [113]:
15+22

37

In [112]:
len(num_cols)

15

- Add str to some numeric
    - Try XGB with str achiv how is the score
    
- yeo johnson transformation
- Kmeans / DBSCAn
- Remove Outlier
- Embedding
- Tuning
- Rule based for more specialized model
- Using no Add fe and ensemble of add _fe etc